## Project Continuous Control - Report

###  DDPG  Algorithm
This project uses _Algorithm DDPG_(_Deep Deterministic Policy Gradient_)
DDPG learn Q functions and policies
at the same time. This method uses off policy data and the Bellman equation to learn Q function and similarly use Q function to learn the policy. This dual mechanism is called _actor-critic method_. Furthermore, The DDPG algorithm uses two additional mechanisms: _replay buffer_ and _soft update_.


### Goal of DDPG Agent 
The environment of this project is to control the **double-joint arm** to reach the target position.
For each step where the agent's hand is at the global location, a reward of +0.1 will be provided. Therefore,
The agent must keep his position at the target location for as many steps as possible.

Observation space has 33 dimensions according to the position, rotation, speed,
And the angular velocity of the arm. There are 4 dimensions in the action space which are correspoding to torque applicable to
Two joints. Each item in the motion vector must be a number between -1 and 1.



### Target networks

This is the target network used for slow tracking of the trained network. We make a copy of the _actor_ and _critic_ networks.
_actor_\__target_ (e.g. using parameter vector _p'_) and _critic _\__ target_ (e.g. using parameter vector _w'_).
These _target networks_ will be updated according to the following tracks.

    p'  <--  p * \tau + p' * (1 - \tau)  
    w'  <--  w * \tau + w' * (1 - \tau)

Enter a very small value for _ \ tau _ (= 0.001). This means that the target value is limited to change slowly, which greatly improves the stability of learning. This update is performed by the _soft _ \ __ update_ function.


### DDPG Neural Networks

The DDPG algorithm uses 4 neural networks: _actor_\__target_, _actor_\__local_, _critic_\__target_ and _critic_\__local_:

    actor_local = Actor(state_size, action_size, random_seed).to(device)
    actor_target = Actor(state_size, action_size, random_seed).to(device)

    critic_local = Critic(state_size, action_size, random_seed).to(device)
    critic_target = Critic(state_size, action_size, random_seed).to(device)

The _Actor_ and _Critic_ classes are provided by model.py, which are Typical behavior of actors and critics
is as follows:

    actor_target(state) -> action
    critic_target(state, action) -> Q-value
    
    actor_local(states) -> actions_pred
    -critic_local(states, actions_pred) -> actor_loss

###     Actor-Critic dual mechanism

[image1]: im_1.png  "im1"
[image2]: im_2.png  "im2"
[image3]: im_3.png  "im3"
[image4]: im_4.png  "im4"
[image5]: im_5.png  "im5"
[image6]: im_6.png  "im6"

For each time step _t, Let's proceed the following:<br>
Let *S&nbsp;*as it is at the current state. This is the input of  Actor NN_ . Then, we have the output as action-value.<br>
![](im_1.png)
![im1][image1] 
<h2><center>$\pi(A|S,\theta)$</center></h2>
where \pi is the policy function,  i.e., the distribution of the actions. The _Critic NN_  gets the state __*S&nbsp;*__ as input and outputs the state-value function __*v(S,w)*__ , that is the _expected total reward_ for the agent starting from state __*S&nbsp;*__. Here, _\theta_ is the vector parameter of the _Actor NN_, _w&nbsp;_ - the vector parameter of the _Critic NN_. The task is to train both networks, i.e., to find the optimal values for _\theta_ and _w&nbsp;_.  By policy _\pi_ we get the action _A&nbsp;_,  from the environment we get reward _R&nbsp;_ and the next state __*S'&nbsp;*__. Then we get _TD-estimate_:

![](im_2.png) 
![im2][image2]  
<h2><center>$Q(S,A)=R+\gamma(S,W)$</center></h2>

Succeedingly, we use the _Critic_ to calculate the _advantage function_ _A(s, a)_:

![](im_3.png)
![im2][image2] 
<h2><center>$\delta\leftarrow R + \gamma \hat{v}(S',W) -\hat{v}(S,W) = Q(S',A) - \hat{v}(S,W)$</center></h2>
				 
At this time, _\gamma_ is the _discount factor_. The parameter _\theta_ is updated by gradient ascent as follows:

![](im_4.png)
![im3][image3]
<h2><center>$\theta \leftarrow \theta + \alpha I\delta\nabla_{\theta}log\pi(A|S,\theta)$</center></h2>

The parameter _w&nbsp;_ is updated as follows:

![](im_5.png)
![im4][image4]
<h2><center>$w\leftarrow w + \beta\nabla_{w}\hat{v}(S,W)$</center></h2><br>
At this point, \alpha (resp. \beta) is the learning rate for the _Actor NN_ (resp. _Critic NN_).  Before we return to the next timestep we update the state _S&nbsp;_ and the operator _I&nbsp;_ by _discount factor_ \gamma:

![](im_6.png)
![im5][image5]
<h2><center>$I\leftarrow\gamma I, S\leftarrow S'$</center></h2><br>
,
In the beginning of the algorithm the operator, _I_ should be initialized to the identity opeartor. 

### Update critic_local neural network in pseudocode
  
     1. Get predicted next-state actions and Q-values from the actor and critic target neural networks.
        actions_next = actor_target(next_states)
        Q_targets_next = critic_target(next_states, actions_next)

     2. Compute Q-targets by for current states (by Bellman equation)
        Q_targets = rewards + (gamma * Q_targets_next * (1 - dones))

     3. Compute Q_expected and critic loss
        Q_expected = critic_local(states, actions)
        critic_loss = MSE_loss(Q_expected, Q_targets)

     4. Minimize the critic_loss. By Gradient Descent and Backward Propogation the weights of 
        the critic_local network are updated. 
        
### Update actor_local neural network in pseudocode 

    1. Compute actor loss
        actions_pred = actor_local(states)
        actor_loss = -critic_local(states, actions_pred).mean()

    2. Minimize the actor_loss. By Gradient Descent and Backward Propogation the weights of 
       the actor_local network are updated.
       
 See method _learn()_ in  _ddpg_\__agent.py_  
 
### Architecture of the actor and critic networks

Both the _actor_ and _critic_ classes implement the neural network    
with 3 fully-connected layers and 2 rectified nonlinear layers. These networks are realized in the framework   
of package PyTorch. Such a network is used in Udacity _model.py_ code for the Pendulum model using DDPG.   
The number of neurons of the fully-connected layers are as follows:

for the _actor_:   
Layer fc1, number of neurons: state_size x fc1_units,   
Layer fc2, number of neurons: fc1_units x fc2_units,    
Layer fc3, number of neurons: fc2_units x action_size,

for the _critic_:   
Layer fcs1, number of neurons: state_size x fcs1_units,  
Layer fc2, number of neurons: (fcs1_units+action_size) x fc2_units,   
Layer fc3, number of neurons: fc2_units x 1. 

Here, state_size = 33, action_size = 4. The input parameters fc1_units, fc2_units, fcs1_units are all taken = 128. 


### Hyperparameters

     BUFFER_SIZE = int(1e6)  # replay buffer size    
     BATCH_SIZE = 256        # minibatch size    
     GAMMA = 0.99            # discount factor    
     TAU = 1e-3              # for soft update of target parameters   
     LR_ACTOR = 1e-3         # learning rate of the actor    
     LR_CRITIC = 1e-3        # learning rate of the critic   
     WEIGHT_DECAY = 0        # L2 weight decay   
     EPSILON = 1.0           # epsilon noise parameter   
     EPSILON_DECAY = 1e-6    # decay parameter of epsilon    
     LEARNING_PERIOD = 20    # learning frequency      
     UPDATE_FACTOR   = 10    # how much to learn    
     
Note that parameters LEARNING_PERIOD and UPDATE_FACTOR are critical for the **convergence** of the algorithm.    
The corresponding code is in the function _step()_.    
     
     if len(self.memory) > BATCH_SIZE and timestep % LEARNING_PERIOD == 0:
            for _ in range(UPDATE_FACTOR):
                experiences = self.memory.sample()
                self.learn(experiences, GAMMA)


### Training the Agent
[image7]: result.png  "im7"
![](result.png)
![im7][image7]<br>
On the Udacity-provied GPU enviroment, the desired average reward was achieved over 1000 episodes in almost 6 hours.


### Full log over all episodes

Episode: 1, Score: 0.00, Max: 0.00, Min: 0.00   
Episode: 2, Score: 1.00, Max: 1.00, Min: 1.00  
Episode: 3, Score: 0.32, Max: 0.32, Min: 0.32   
Episode: 4, Score: 0.88, Max: 0.88, Min: 0.88   
Episode: 5, Score: 0.49, Max: 0.49, Min: 0.49   
Episode: 6, Score: 0.13, Max: 0.13, Min: 0.13   
Episode: 7, Score: 0.77, Max: 0.77, Min: 0.77   
Episode: 8, Score: 1.37, Max: 1.37, Min: 1.37   
Episode: 9, Score: 0.51, Max: 0.51, Min: 0.51   
Episode: 10, Score: 0.58, Max: 0.58, Min: 0.58   
*** Episode 10	Average Score: 0.60, Time: 00:01:24 ***   
Episode: 11, Score: 1.46, Max: 1.46, Min: 1.46     
Episode: 12, Score: 1.65, Max: 1.65, Min: 1.65   
Episode: 13, Score: 1.15, Max: 1.15, Min: 1.15   
Episode: 14, Score: 0.59, Max: 0.59, Min: 0.59   
Episode: 15, Score: 2.26, Max: 2.26, Min: 2.26   
Episode: 16, Score: 1.35, Max: 1.35, Min: 1.35   
Episode: 17, Score: 0.72, Max: 0.72, Min: 0.72   
Episode: 18, Score: 0.96, Max: 0.96, Min: 0.96   
Episode: 19, Score: 1.77, Max: 1.77, Min: 1.77   
Episode: 20, Score: 1.44, Max: 1.44, Min: 1.44   
*** Episode 20	Average Score: 0.97, Time: 00:02:51 ***  
Episode: 21, Score: 2.05, Max: 2.05, Min: 2.05   
Episode: 22, Score: 2.31, Max: 2.31, Min: 2.31   
Episode: 23, Score: 2.72, Max: 2.72, Min: 2.72   
Episode: 24, Score: 2.15, Max: 2.15, Min: 2.15   
Episode: 25, Score: 3.72, Max: 3.72, Min: 3.72   
Episode: 26, Score: 2.48, Max: 2.48, Min: 2.48   
Episode: 27, Score: 3.33, Max: 3.33, Min: 3.33    
Episode: 28, Score: 1.53, Max: 1.53, Min: 1.53  
Episode: 29, Score: 4.03, Max: 4.03, Min: 4.03   
Episode: 30, Score: 1.79, Max: 1.79, Min: 1.79   
*** Episode 30	Average Score: 1.52, Time: 00:04:18 ***   
Episode: 31, Score: 1.13, Max: 1.13, Min: 1.13  
Episode: 32, Score: 2.48, Max: 2.48, Min: 2.48  
Episode: 33, Score: 1.79, Max: 1.79, Min: 1.79  
Episode: 34, Score: 2.54, Max: 2.54, Min: 2.54  
Episode: 35, Score: 3.62, Max: 3.62, Min: 3.62  
Episode: 36, Score: 0.34, Max: 0.34, Min: 0.34  
Episode: 37, Score: 2.89, Max: 2.89, Min: 2.89  
Episode: 38, Score: 3.06, Max: 3.06, Min: 3.06  
Episode: 39, Score: 3.61, Max: 3.61, Min: 3.61  
Episode: 40, Score: 1.39, Max: 1.39, Min: 1.39  
*** Episode 40	Average Score: 1.71, Time: 00:05:46 ***  
Episode: 41, Score: 1.29, Max: 1.29, Min: 1.29  
Episode: 42, Score: 1.70, Max: 1.70, Min: 1.70  
Episode: 43, Score: 3.24, Max: 3.24, Min: 3.24  
Episode: 44, Score: 2.32, Max: 2.32, Min: 2.32  
Episode: 45, Score: 1.24, Max: 1.24, Min: 1.24  
Episode: 46, Score: 1.35, Max: 1.35, Min: 1.35  
Episode: 47, Score: 2.42, Max: 2.42, Min: 2.42  
Episode: 48, Score: 3.94, Max: 3.94, Min: 3.94  
Episode: 49, Score: 3.16, Max: 3.16, Min: 3.16  
Episode: 50, Score: 3.56, Max: 3.56, Min: 3.56  
*** Episode 50	Average Score: 1.85, Time: 00:07:14 ***  
Episode: 51, Score: 2.80, Max: 2.80, Min: 2.80  
Episode: 52, Score: 2.96, Max: 2.96, Min: 2.96  
Episode: 53, Score: 2.06, Max: 2.06, Min: 2.06  
Episode: 54, Score: 3.16, Max: 3.16, Min: 3.16  
Episode: 55, Score: 3.58, Max: 3.58, Min: 3.58  
Episode: 56, Score: 4.62, Max: 4.62, Min: 4.62  
Episode: 57, Score: 2.39, Max: 2.39, Min: 2.39  
Episode: 58, Score: 3.58, Max: 3.58, Min: 3.58  
Episode: 59, Score: 2.32, Max: 2.32, Min: 2.32  
Episode: 60, Score: 2.78, Max: 2.78, Min: 2.78  
*** Episode 60	Average Score: 2.05, Time: 00:08:42 ***  
Episode: 61, Score: 2.88, Max: 2.88, Min: 2.88  
Episode: 62, Score: 3.78, Max: 3.78, Min: 3.78  
Episode: 63, Score: 6.23, Max: 6.23, Min: 6.23  
Episode: 64, Score: 3.86, Max: 3.86, Min: 3.86  
Episode: 65, Score: 6.07, Max: 6.07, Min: 6.07  
Episode: 66, Score: 5.18, Max: 5.18, Min: 5.18  
Episode: 67, Score: 2.40, Max: 2.40, Min: 2.40  
Episode: 68, Score: 3.13, Max: 3.13, Min: 3.13  
Episode: 69, Score: 2.48, Max: 2.48, Min: 2.48  
Episode: 70, Score: 4.23, Max: 4.23, Min: 4.23  
*** Episode 70	Average Score: 2.33, Time: 00:10:09 ***  
Episode: 71, Score: 2.89, Max: 2.89, Min: 2.89  
Episode: 72, Score: 2.77, Max: 2.77, Min: 2.77  
Episode: 73, Score: 2.18, Max: 2.18, Min: 2.18  
Episode: 74, Score: 6.77, Max: 6.77, Min: 6.77  
Episode: 75, Score: 3.22, Max: 3.22, Min: 3.22  
Episode: 76, Score: 4.78, Max: 4.78, Min: 4.78  
Episode: 77, Score: 4.63, Max: 4.63, Min: 4.63  
Episode: 78, Score: 2.68, Max: 2.68, Min: 2.68  
Episode: 79, Score: 2.60, Max: 2.60, Min: 2.60  
Episode: 80, Score: 5.15, Max: 5.15, Min: 5.15  
*** Episode 80	Average Score: 2.51, Time: 00:11:38 ***  
Episode: 81, Score: 2.10, Max: 2.10, Min: 2.10  
Episode: 82, Score: 5.66, Max: 5.66, Min: 5.66  
Episode: 83, Score: 2.04, Max: 2.04, Min: 2.04  
Episode: 84, Score: 1.87, Max: 1.87, Min: 1.87  
Episode: 85, Score: 5.98, Max: 5.98, Min: 5.98  
Episode: 86, Score: 4.03, Max: 4.03, Min: 4.03  
Episode: 87, Score: 3.19, Max: 3.19, Min: 3.19  
Episode: 88, Score: 3.24, Max: 3.24, Min: 3.24  
Episode: 89, Score: 2.30, Max: 2.30, Min: 2.30  
Episode: 90, Score: 3.07, Max: 3.07, Min: 3.07   
*** Episode 90	Average Score: 2.60, Time: 00:13:06 ***  
Episode: 91, Score: 4.81, Max: 4.81, Min: 4.81  
Episode: 92, Score: 7.47, Max: 7.47, Min: 7.47  
Episode: 93, Score: 3.16, Max: 3.16, Min: 3.16  
Episode: 94, Score: 3.42, Max: 3.42, Min: 3.42  
Episode: 95, Score: 1.90, Max: 1.90, Min: 1.90  
Episode: 96, Score: 2.98, Max: 2.98, Min: 2.98  
Episode: 97, Score: 2.81, Max: 2.81, Min: 2.81  
Episode: 98, Score: 5.32, Max: 5.32, Min: 5.32  
Episode: 99, Score: 4.18, Max: 4.18, Min: 4.18  
Episode: 100, Score: 3.44, Max: 3.44, Min: 3.44  
*** Episode 100	Average Score: 2.74, Time: 00:14:35 ***  
Episode: 101, Score: 2.95, Max: 2.95, Min: 2.95  
Episode: 102, Score: 3.84, Max: 3.84, Min: 3.84  
Episode: 103, Score: 2.61, Max: 2.61, Min: 2.61  
Episode: 104, Score: 3.66, Max: 3.66, Min: 3.66  
Episode: 105, Score: 5.18, Max: 5.18, Min: 5.18  
Episode: 106, Score: 1.21, Max: 1.21, Min: 1.21  
Episode: 107, Score: 2.71, Max: 2.71, Min: 2.71  
Episode: 108, Score: 2.70, Max: 2.70, Min: 2.70  
Episode: 109, Score: 3.42, Max: 3.42, Min: 3.42  
Episode: 110, Score: 3.07, Max: 3.07, Min: 3.07  
*** Episode 110	Average Score: 2.99, Time: 00:16:04 ***  
Episode: 111, Score: 5.51, Max: 5.51, Min: 5.51  
Episode: 112, Score: 3.65, Max: 3.65, Min: 3.65  
Episode: 113, Score: 3.87, Max: 3.87, Min: 3.87  
Episode: 114, Score: 2.25, Max: 2.25, Min: 2.25  
Episode: 115, Score: 3.71, Max: 3.71, Min: 3.71  
Episode: 116, Score: 3.66, Max: 3.66, Min: 3.66  
Episode: 117, Score: 1.95, Max: 1.95, Min: 1.95  
Episode: 118, Score: 5.34, Max: 5.34, Min: 5.34  
Episode: 119, Score: 2.74, Max: 2.74, Min: 2.74  
Episode: 120, Score: 3.17, Max: 3.17, Min: 3.17  
*** Episode 120	Average Score: 3.22, Time: 00:17:34 ***  
Episode: 121, Score: 1.69, Max: 1.69, Min: 1.69  
Episode: 122, Score: 3.26, Max: 3.26, Min: 3.26  
Episode: 123, Score: 3.61, Max: 3.61, Min: 3.61  
Episode: 124, Score: 2.86, Max: 2.86, Min: 2.86  
Episode: 125, Score: 4.32, Max: 4.32, Min: 4.32  
Episode: 126, Score: 3.07, Max: 3.07, Min: 3.07  
Episode: 127, Score: 2.39, Max: 2.39, Min: 2.39  
Episode: 128, Score: 5.08, Max: 5.08, Min: 5.08  
Episode: 129, Score: 2.55, Max: 2.55, Min: 2.55  
Episode: 130, Score: 4.01, Max: 4.01, Min: 4.01  
*** Episode 130	Average Score: 3.28, Time: 00:19:03 ***  
Episode: 131, Score: 4.03, Max: 4.03, Min: 4.03  
Episode: 132, Score: 3.04, Max: 3.04, Min: 3.04  
Episode: 133, Score: 2.19, Max: 2.19, Min: 2.19  
Episode: 134, Score: 1.44, Max: 1.44, Min: 1.44  
Episode: 135, Score: 3.27, Max: 3.27, Min: 3.27  
Episode: 136, Score: 3.23, Max: 3.23, Min: 3.23  
Episode: 137, Score: 2.03, Max: 2.03, Min: 2.03  
Episode: 138, Score: 2.95, Max: 2.95, Min: 2.95  
Episode: 139, Score: 3.93, Max: 3.93, Min: 3.93  
Episode: 140, Score: 3.45, Max: 3.45, Min: 3.45  
*** Episode 140	Average Score: 3.35, Time: 00:20:33 ***  
Episode: 141, Score: 3.14, Max: 3.14, Min: 3.14  
Episode: 142, Score: 2.55, Max: 2.55, Min: 2.55  
Episode: 143, Score: 3.73, Max: 3.73, Min: 3.73  
Episode: 144, Score: 3.90, Max: 3.90, Min: 3.90  
Episode: 145, Score: 3.69, Max: 3.69, Min: 3.69  
Episode: 146, Score: 2.48, Max: 2.48, Min: 2.48  
Episode: 147, Score: 4.10, Max: 4.10, Min: 4.10  
Episode: 148, Score: 10.17, Max: 10.17, Min: 10.17  
Episode: 149, Score: 3.21, Max: 3.21, Min: 3.21  
Episode: 150, Score: 3.51, Max: 3.51, Min: 3.51  
*** Episode 150	Average Score: 3.51, Time: 00:22:03 ***  
Episode: 151, Score: 6.22, Max: 6.22, Min: 6.22  
Episode: 152, Score: 3.62, Max: 3.62, Min: 3.62  
Episode: 153, Score: 3.73, Max: 3.73, Min: 3.73  
Episode: 154, Score: 4.06, Max: 4.06, Min: 4.06  
Episode: 155, Score: 3.57, Max: 3.57, Min: 3.57  
Episode: 156, Score: 3.60, Max: 3.60, Min: 3.60  
Episode: 157, Score: 4.41, Max: 4.41, Min: 4.41  
Episode: 158, Score: 2.79, Max: 2.79, Min: 2.79  
Episode: 159, Score: 6.89, Max: 6.89, Min: 6.89  
Episode: 160, Score: 4.43, Max: 4.43, Min: 4.43  
*** Episode 160	Average Score: 3.64, Time: 00:23:34 ***  
Episode: 161, Score: 4.40, Max: 4.40, Min: 4.40   
Episode: 162, Score: 7.23, Max: 7.23, Min: 7.23  
Episode: 163, Score: 3.65, Max: 3.65, Min: 3.65  
Episode: 164, Score: 4.25, Max: 4.25, Min: 4.25  
Episode: 165, Score: 7.35, Max: 7.35, Min: 7.35  
Episode: 166, Score: 4.52, Max: 4.52, Min: 4.52  
Episode: 167, Score: 5.74, Max: 5.74, Min: 5.74  
Episode: 168, Score: 4.81, Max: 4.81, Min: 4.81  
Episode: 169, Score: 7.11, Max: 7.11, Min: 7.11  
Episode: 170, Score: 3.64, Max: 3.64, Min: 3.64   
*** Episode 170	Average Score: 3.77, Time: 00:25:05 ***  
Episode: 171, Score: 4.02, Max: 4.02, Min: 4.02  
Episode: 172, Score: 1.45, Max: 1.45, Min: 1.45  
Episode: 173, Score: 4.61, Max: 4.61, Min: 4.61  
Episode: 174, Score: 3.35, Max: 3.35, Min: 3.35  
Episode: 175, Score: 4.99, Max: 4.99, Min: 4.99  
Episode: 176, Score: 4.83, Max: 4.83, Min: 4.83  
Episode: 177, Score: 3.31, Max: 3.31, Min: 3.31  
Episode: 178, Score: 6.25, Max: 6.25, Min: 6.25  
Episode: 179, Score: 3.46, Max: 3.46, Min: 3.46  
Episode: 180, Score: 3.40, Max: 3.40, Min: 3.40   
*** Episode 180	Average Score: 3.79, Time: 00:26:37 ***   
Episode: 181, Score: 4.83, Max: 4.83, Min: 4.83  
Episode: 182, Score: 4.30, Max: 4.30, Min: 4.30  
Episode: 183, Score: 3.08, Max: 3.08, Min: 3.08  
Episode: 184, Score: 4.16, Max: 4.16, Min: 4.16  
Episode: 185, Score: 4.06, Max: 4.06, Min: 4.06  
Episode: 186, Score: 4.05, Max: 4.05, Min: 4.05  
Episode: 187, Score: 3.09, Max: 3.09, Min: 3.09  
Episode: 188, Score: 5.07, Max: 5.07, Min: 5.07   
Episode: 189, Score: 3.08, Max: 3.08, Min: 3.08   
Episode: 190, Score: 2.63, Max: 2.63, Min: 2.63  
*** Episode 190	Average Score: 3.84, Time: 00:28:08 ***  
Episode: 191, Score: 5.35, Max: 5.35, Min: 5.35  
Episode: 192, Score: 2.80, Max: 2.80, Min: 2.80  
Episode: 193, Score: 5.41, Max: 5.41, Min: 5.41   
Episode: 194, Score: 4.17, Max: 4.17, Min: 4.17   
Episode: 195, Score: 5.94, Max: 5.94, Min: 5.94   
Episode: 196, Score: 4.66, Max: 4.66, Min: 4.66   
Episode: 197, Score: 4.58, Max: 4.58, Min: 4.58   
Episode: 198, Score: 3.39, Max: 3.39, Min: 3.39   
Episode: 199, Score: 3.76, Max: 3.76, Min: 3.76   
Episode: 200, Score: 5.83, Max: 5.83, Min: 5.83       
*** Episode 200	Average Score: 3.90, Time: 00:29:40 ***  
Episode: 201, Score: 4.01, Max: 4.01, Min: 4.01   
Episode: 202, Score: 4.80, Max: 4.80, Min: 4.80  
Episode: 203, Score: 3.70, Max: 3.70, Min: 3.70  
Episode: 204, Score: 2.33, Max: 2.33, Min: 2.33  
Episode: 205, Score: 6.06, Max: 6.06, Min: 6.06  
Episode: 206, Score: 3.01, Max: 3.01, Min: 3.01  
Episode: 207, Score: 4.45, Max: 4.45, Min: 4.45  
Episode: 208, Score: 3.82, Max: 3.82, Min: 3.82  
Episode: 209, Score: 2.94, Max: 2.94, Min: 2.94  
Episode: 210, Score: 3.41, Max: 3.41, Min: 3.41  
*** Episode 210	Average Score: 3.97, Time: 00:31:13 ***  
Episode: 211, Score: 3.84, Max: 3.84, Min: 3.84  
Episode: 212, Score: 4.48, Max: 4.48, Min: 4.48  
Episode: 213, Score: 5.68, Max: 5.68, Min: 5.68  
Episode: 214, Score: 4.71, Max: 4.71, Min: 4.71  
Episode: 215, Score: 2.09, Max: 2.09, Min: 2.09  
Episode: 216, Score: 4.82, Max: 4.82, Min: 4.82  
Episode: 217, Score: 6.21, Max: 6.21, Min: 6.21  
Episode: 218, Score: 2.74, Max: 2.74, Min: 2.74  
Episode: 219, Score: 2.41, Max: 2.41, Min: 2.41  
Episode: 220, Score: 4.62, Max: 4.62, Min: 4.62  
*** Episode 220	Average Score: 4.03, Time: 00:32:46 ***  
Episode: 221, Score: 4.25, Max: 4.25, Min: 4.25  
Episode: 222, Score: 3.93, Max: 3.93, Min: 3.93  
Episode: 223, Score: 4.44, Max: 4.44, Min: 4.44  
Episode: 224, Score: 4.10, Max: 4.10, Min: 4.10  
Episode: 225, Score: 3.85, Max: 3.85, Min: 3.85  
Episode: 226, Score: 3.75, Max: 3.75, Min: 3.75  
Episode: 227, Score: 5.92, Max: 5.92, Min: 5.92  
Episode: 228, Score: 4.67, Max: 4.67, Min: 4.67  
Episode: 229, Score: 3.57, Max: 3.57, Min: 3.57  
Episode: 230, Score: 2.53, Max: 2.53, Min: 2.53  
*** Episode 230	Average Score: 4.11, Time: 00:34:20 ***  
Episode: 231, Score: 3.54, Max: 3.54, Min: 3.54  
Episode: 232, Score: 5.50, Max: 5.50, Min: 5.50  
Episode: 233, Score: 4.54, Max: 4.54, Min: 4.54  
Episode: 234, Score: 6.12, Max: 6.12, Min: 6.12  
Episode: 235, Score: 3.94, Max: 3.94, Min: 3.94  
Episode: 236, Score: 6.10, Max: 6.10, Min: 6.10  
Episode: 237, Score: 5.48, Max: 5.48, Min: 5.48  
Episode: 238, Score: 5.73, Max: 5.73, Min: 5.73  
Episode: 239, Score: 5.95, Max: 5.95, Min: 5.95  
Episode: 240, Score: 4.77, Max: 4.77, Min: 4.77  
*** Episode 240	Average Score: 4.33, Time: 00:35:55 ***  
Episode: 241, Score: 3.57, Max: 3.57, Min: 3.57  
Episode: 242, Score: 3.63, Max: 3.63, Min: 3.63  
Episode: 243, Score: 4.50, Max: 4.50, Min: 4.50  
Episode: 244, Score: 6.16, Max: 6.16, Min: 6.16  
Episode: 245, Score: 2.44, Max: 2.44, Min: 2.44  
Episode: 246, Score: 8.11, Max: 8.11, Min: 8.11  
Episode: 247, Score: 3.29, Max: 3.29, Min: 3.29  
Episode: 248, Score: 2.95, Max: 2.95, Min: 2.95  
Episode: 249, Score: 3.98, Max: 3.98, Min: 3.98  
Episode: 250, Score: 1.54, Max: 1.54, Min: 1.54   
*** Episode 250	Average Score: 4.33, Time: 00:37:31 ***  
Episode: 251, Score: 5.57, Max: 5.57, Min: 5.57  
Episode: 252, Score: 4.68, Max: 4.68, Min: 4.68  
Episode: 253, Score: 6.84, Max: 6.84, Min: 6.84  
Episode: 254, Score: 8.38, Max: 8.38, Min: 8.38  
Episode: 255, Score: 5.02, Max: 5.02, Min: 5.02  
Episode: 256, Score: 5.35, Max: 5.35, Min: 5.35  
Episode: 257, Score: 5.25, Max: 5.25, Min: 5.25  
Episode: 258, Score: 4.06, Max: 4.06, Min: 4.06  
Episode: 259, Score: 8.18, Max: 8.18, Min: 8.18  
Episode: 260, Score: 7.16, Max: 7.16, Min: 7.16  
*** Episode 260	Average Score: 4.50, Time: 00:39:07 ***  
Episode: 261, Score: 5.16, Max: 5.16, Min: 5.16  
Episode: 262, Score: 4.79, Max: 4.79, Min: 4.79  
Episode: 263, Score: 4.38, Max: 4.38, Min: 4.38  
Episode: 264, Score: 6.99, Max: 6.99, Min: 6.99  
Episode: 265, Score: 5.56, Max: 5.56, Min: 5.56  
Episode: 266, Score: 9.32, Max: 9.32, Min: 9.32  
Episode: 267, Score: 5.17, Max: 5.17, Min: 5.17  
Episode: 268, Score: 3.79, Max: 3.79, Min: 3.79  
Episode: 269, Score: 5.58, Max: 5.58, Min: 5.58   
Episode: 270, Score: 5.16, Max: 5.16, Min: 5.16  
*** Episode 270	Average Score: 4.53, Time: 00:40:44 ***  
Episode: 271, Score: 2.74, Max: 2.74, Min: 2.74  
Episode: 272, Score: 8.01, Max: 8.01, Min: 8.01  
Episode: 273, Score: 3.12, Max: 3.12, Min: 3.12  
Episode: 274, Score: 6.46, Max: 6.46, Min: 6.46  
Episode: 275, Score: 3.47, Max: 3.47, Min: 3.47  
Episode: 276, Score: 6.54, Max: 6.54, Min: 6.54  
Episode: 277, Score: 4.45, Max: 4.45, Min: 4.45  
Episode: 278, Score: 4.49, Max: 4.49, Min: 4.49  
Episode: 279, Score: 6.72, Max: 6.72, Min: 6.72  
Episode: 280, Score: 4.74, Max: 4.74, Min: 4.74   
*** Episode 280	Average Score: 4.64, Time: 00:42:22 ***  
Episode: 281, Score: 5.85, Max: 5.85, Min: 5.85  
Episode: 282, Score: 3.94, Max: 3.94, Min: 3.94  
Episode: 283, Score: 2.97, Max: 2.97, Min: 2.97  
Episode: 284, Score: 6.47, Max: 6.47, Min: 6.47  
Episode: 285, Score: 4.53, Max: 4.53, Min: 4.53  
Episode: 286, Score: 1.44, Max: 1.44, Min: 1.44  
Episode: 287, Score: 2.65, Max: 2.65, Min: 2.65  
Episode: 288, Score: 6.10, Max: 6.10, Min: 6.10  
Episode: 289, Score: 4.36, Max: 4.36, Min: 4.36  
Episode: 290, Score: 3.71, Max: 3.71, Min: 3.71  
*** Episode 290	Average Score: 4.68, Time: 00:44:01 ***  
Episode: 291, Score: 5.70, Max: 5.70, Min: 5.70  
Episode: 292, Score: 5.46, Max: 5.46, Min: 5.46  
Episode: 293, Score: 4.85, Max: 4.85, Min: 4.85  
Episode: 294, Score: 3.54, Max: 3.54, Min: 3.54  
Episode: 295, Score: 6.18, Max: 6.18, Min: 6.18  
Episode: 296, Score: 3.16, Max: 3.16, Min: 3.16  
Episode: 297, Score: 5.31, Max: 5.31, Min: 5.31  
Episode: 298, Score: 5.17, Max: 5.17, Min: 5.17  
Episode: 299, Score: 4.25, Max: 4.25, Min: 4.25  
Episode: 300, Score: 3.88, Max: 3.88, Min: 3.88  
*** Episode 300	Average Score: 4.70, Time: 00:45:41 ***  
Episode: 301, Score: 7.73, Max: 7.73, Min: 7.73  
Episode: 302, Score: 5.39, Max: 5.39, Min: 5.39  
Episode: 303, Score: 10.71, Max: 10.71, Min: 10.71  
Episode: 304, Score: 9.39, Max: 9.39, Min: 9.39  
Episode: 305, Score: 3.42, Max: 3.42, Min: 3.42  
Episode: 306, Score: 9.37, Max: 9.37, Min: 9.37  
Episode: 307, Score: 4.76, Max: 4.76, Min: 4.76  
Episode: 308, Score: 5.03, Max: 5.03, Min: 5.03  
Episode: 309, Score: 7.96, Max: 7.96, Min: 7.96  
Episode: 310, Score: 5.51, Max: 5.51, Min: 5.51  
*** Episode 310	Average Score: 5.00, Time: 00:47:21 ***  
Episode: 311, Score: 9.22, Max: 9.22, Min: 9.22  
Episode: 312, Score: 5.75, Max: 5.75, Min: 5.75  
Episode: 313, Score: 5.99, Max: 5.99, Min: 5.99  
Episode: 314, Score: 4.49, Max: 4.49, Min: 4.49  
Episode: 315, Score: 6.81, Max: 6.81, Min: 6.81  
Episode: 316, Score: 5.61, Max: 5.61, Min: 5.61  
Episode: 317, Score: 4.25, Max: 4.25, Min: 4.25  
Episode: 318, Score: 4.00, Max: 4.00, Min: 4.00  
Episode: 319, Score: 5.47, Max: 5.47, Min: 5.47  
Episode: 320, Score: 7.97, Max: 7.97, Min: 7.97   
*** Episode 320	Average Score: 5.18, Time: 00:49:03 ***  
Episode: 321, Score: 5.68, Max: 5.68, Min: 5.68  
Episode: 322, Score: 10.84, Max: 10.84, Min: 10.84  
Episode: 323, Score: 5.57, Max: 5.57, Min: 5.57  
Episode: 324, Score: 6.05, Max: 6.05, Min: 6.05  
Episode: 325, Score: 5.89, Max: 5.89, Min: 5.89  
Episode: 326, Score: 10.38, Max: 10.38, Min: 10.38  
Episode: 327, Score: 6.87, Max: 6.87, Min: 6.87  
Episode: 328, Score: 8.75, Max: 8.75, Min: 8.75  
Episode: 329, Score: 6.36, Max: 6.36, Min: 6.36  
Episode: 330, Score: 4.29, Max: 4.29, Min: 4.29  
*** Episode 330	Average Score: 5.48, Time: 00:50:46 ***  
Episode: 331, Score: 7.35, Max: 7.35, Min: 7.35  
Episode: 332, Score: 5.93, Max: 5.93, Min: 5.93  
Episode: 333, Score: 4.00, Max: 4.00, Min: 4.00  
Episode: 334, Score: 5.93, Max: 5.93, Min: 5.93  
Episode: 335, Score: 4.68, Max: 4.68, Min: 4.68  
Episode: 336, Score: 4.09, Max: 4.09, Min: 4.09  
Episode: 337, Score: 6.10, Max: 6.10, Min: 6.10  
Episode: 338, Score: 5.00, Max: 5.00, Min: 5.00  
Episode: 339, Score: 4.99, Max: 4.99, Min: 4.99  
Episode: 340, Score: 4.40, Max: 4.40, Min: 4.40  
*** Episode 340	Average Score: 5.49, Time: 00:52:31 ***  
Episode: 341, Score: 3.98, Max: 3.98, Min: 3.98  
Episode: 342, Score: 10.07, Max: 10.07, Min: 10.07  
Episode: 343, Score: 4.59, Max: 4.59, Min: 4.59  
Episode: 344, Score: 2.61, Max: 2.61, Min: 2.61  
Episode: 345, Score: 8.45, Max: 8.45, Min: 8.45  
Episode: 346, Score: 7.57, Max: 7.57, Min: 7.57  
Episode: 347, Score: 4.52, Max: 4.52, Min: 4.52  
Episode: 348, Score: 6.37, Max: 6.37, Min: 6.37  
Episode: 349, Score: 5.54, Max: 5.54, Min: 5.54  
Episode: 350, Score: 4.38, Max: 4.38, Min: 4.38   
*** Episode 350	Average Score: 5.67, Time: 00:54:16 ***  
Episode: 351, Score: 6.62, Max: 6.62, Min: 6.62  
Episode: 352, Score: 4.67, Max: 4.67, Min: 4.67  
Episode: 353, Score: 10.47, Max: 10.47, Min: 10.47  
Episode: 354, Score: 7.56, Max: 7.56, Min: 7.56  
Episode: 355, Score: 7.87, Max: 7.87, Min: 7.87  
Episode: 356, Score: 10.92, Max: 10.92, Min: 10.92  
Episode: 357, Score: 6.79, Max: 6.79, Min: 6.79  
Episode: 358, Score: 3.33, Max: 3.33, Min: 3.33  
Episode: 359, Score: 6.72, Max: 6.72, Min: 6.72  
Episode: 360, Score: 6.10, Max: 6.10, Min: 6.10  
*** Episode 360	Average Score: 5.77, Time: 00:56:03 ***  
Episode: 361, Score: 5.32, Max: 5.32, Min: 5.32  
Episode: 362, Score: 4.40, Max: 4.40, Min: 4.40  
Episode: 363, Score: 7.34, Max: 7.34, Min: 7.34  
Episode: 364, Score: 3.62, Max: 3.62, Min: 3.62  
Episode: 365, Score: 11.58, Max: 11.58, Min: 11.58  
Episode: 366, Score: 16.08, Max: 16.08, Min: 16.08  
Episode: 367, Score: 10.67, Max: 10.67, Min: 10.67  
Episode: 368, Score: 6.42, Max: 6.42, Min: 6.42  
Episode: 369, Score: 4.94, Max: 4.94, Min: 4.94  
Episode: 370, Score: 4.90, Max: 4.90, Min: 4.90  
*** Episode 370	Average Score: 5.97, Time: 00:57:51 ***  
Episode: 371, Score: 15.29, Max: 15.29, Min: 15.29  
Episode: 372, Score: 6.60, Max: 6.60, Min: 6.60  
Episode: 373, Score: 10.55, Max: 10.55, Min: 10.55  
Episode: 374, Score: 7.55, Max: 7.55, Min: 7.55  
Episode: 375, Score: 8.18, Max: 8.18, Min: 8.18  
Episode: 376, Score: 5.56, Max: 5.56, Min: 5.56  
Episode: 377, Score: 7.18, Max: 7.18, Min: 7.18  
Episode: 378, Score: 6.35, Max: 6.35, Min: 6.35  
Episode: 379, Score: 10.42, Max: 10.42, Min: 10.42  
Episode: 380, Score: 1.83, Max: 1.83, Min: 1.83  
*** Episode 380	Average Score: 6.25, Time: 00:59:40 ***  
Episode: 381, Score: 8.76, Max: 8.76, Min: 8.76  
Episode: 382, Score: 6.61, Max: 6.61, Min: 6.61  
Episode: 383, Score: 13.12, Max: 13.12, Min: 13.12  
Episode: 384, Score: 14.23, Max: 14.23, Min: 14.23  
Episode: 385, Score: 3.34, Max: 3.34, Min: 3.34  
Episode: 386, Score: 6.14, Max: 6.14, Min: 6.14  
Episode: 387, Score: 4.29, Max: 4.29, Min: 4.29  
Episode: 388, Score: 1.05, Max: 1.05, Min: 1.05  
Episode: 389, Score: 10.90, Max: 10.90, Min: 10.90  
Episode: 390, Score: 2.68, Max: 2.68, Min: 2.68  
*** Episode 390	Average Score: 6.55, Time: 01:01:30 ***  
Episode: 391, Score: 4.23, Max: 4.23, Min: 4.23  
Episode: 392, Score: 1.04, Max: 1.04, Min: 1.04  
Episode: 393, Score: 7.27, Max: 7.27, Min: 7.27  
Episode: 394, Score: 2.15, Max: 2.15, Min: 2.15  
Episode: 395, Score: 5.03, Max: 5.03, Min: 5.03  
Episode: 396, Score: 7.83, Max: 7.83, Min: 7.83  
Episode: 397, Score: 3.94, Max: 3.94, Min: 3.94  
Episode: 398, Score: 4.95, Max: 4.95, Min: 4.95  
Episode: 399, Score: 5.34, Max: 5.34, Min: 5.34  
Episode: 400, Score: 3.38, Max: 3.38, Min: 3.38  
*** Episode 400	Average Score: 6.52, Time: 01:03: 21 ***  
Episode: 401, Score: 9.35, Max: 9.35, Min: 9.35  
Episode: 402, Score: 5.22, Max: 5.22, Min: 5.22  
Episode: 403, Score: 5.22, Max: 5.22, Min: 5.22  
Episode: 404, Score: 12.66, Max: 12.66, Min: 12.66  
Episode: 405, Score: 8.87, Max: 8.87, Min: 8.87  
Episode: 406, Score: 6.21, Max: 6.21, Min: 6.21  
Episode: 407, Score: 7.36, Max: 7.36, Min: 7.36  
Episode: 408, Score: 7.48, Max: 7.48, Min: 7.48  
Episode: 409, Score: 5.06, Max: 5.06, Min: 5.06  
Episode: 410, Score: 5.48, Max: 5.48, Min: 5.48  
*** Episode 410	Average Score: 6.56, Time: 01:05:14 ***  
Episode: 411, Score: 3.86, Max: 3.86, Min: 3.86  
Episode: 412, Score: 5.55, Max: 5.55, Min: 5.55  
Episode: 413, Score: 7.89, Max: 7.89, Min: 7.89  
Episode: 414, Score: 10.08, Max: 10.08, Min: 10.08  
Episode: 415, Score: 4.44, Max: 4.44, Min: 4.44  
Episode: 416, Score: 6.64, Max: 6.64, Min: 6.64  
Episode: 417, Score: 7.23, Max: 7.23, Min: 7.23  
Episode: 418, Score: 8.34, Max: 8.34, Min: 8.34  
Episode: 419, Score: 4.77, Max: 4.77, Min: 4.77  
Episode: 420, Score: 3.15, Max: 3.15, Min: 3.15  
*** Episode 420	Average Score: 6.58, Time: 01:07:08 ***  
Episode: 421, Score: 10.24, Max: 10.24, Min: 10.24  
Episode: 422, Score: 5.27, Max: 5.27, Min: 5.27  
Episode: 423, Score: 3.35, Max: 3.35, Min: 3.35  
Episode: 424, Score: 8.61, Max: 8.61, Min: 8.61  
Episode: 425, Score: 9.26, Max: 9.26, Min: 9.26  
Episode: 426, Score: 7.51, Max: 7.51, Min: 7.51  
Episode: 427, Score: 10.64, Max: 10.64, Min: 10.64  
Episode: 428, Score: 6.45, Max: 6.45, Min: 6.45  
Episode: 429, Score: 5.93, Max: 5.93, Min: 5.93  
Episode: 430, Score: 5.35, Max: 5.35, Min: 5.35  
*** Episode 430	Average Score: 6.60, Time: 01:09:03 ***  
Episode: 431, Score: 4.87, Max: 4.87, Min: 4.87  
Episode: 432, Score: 2.83, Max: 2.83, Min: 2.83  
Episode: 433, Score: 11.08, Max: 11.08, Min: 11.08  
Episode: 434, Score: 5.14, Max: 5.14, Min: 5.14  
Episode: 435, Score: 13.63, Max: 13.63, Min: 13.63  
Episode: 436, Score: 5.75, Max: 5.75, Min: 5.75  
Episode: 437, Score: 8.51, Max: 8.51, Min: 8.51  
Episode: 438, Score: 4.80, Max: 4.80, Min: 4.80  
Episode: 439, Score: 5.02, Max: 5.02, Min: 5.02  
Episode: 440, Score: 6.10, Max: 6.10, Min: 6.10  
*** Episode 440	Average Score: 6.75, Time: 01:11:00 ***  
Episode: 441, Score: 6.41, Max: 6.41, Min: 6.41   
Episode: 442, Score: 5.92, Max: 5.92, Min: 5.92  
Episode: 443, Score: 8.62, Max: 8.62, Min: 8.62  
Episode: 444, Score: 2.56, Max: 2.56, Min: 2.56  
Episode: 445, Score: 8.53, Max: 8.53, Min: 8.53  
Episode: 446, Score: 6.07, Max: 6.07, Min: 6.07  
Episode: 447, Score: 11.01, Max: 11.01, Min: 11.01  
Episode: 448, Score: 7.61, Max: 7.61, Min: 7.61  
Episode: 449, Score: 11.99, Max: 11.99, Min: 11.99   
Episode: 450, Score: 10.71, Max: 10.71, Min: 10.71   
*** Episode 450	Average Score: 6.97, Time: 01:12:58 ***   
Episode: 451, Score: 9.77, Max: 9.77, Min: 9.77   
Episode: 452, Score: 10.97, Max: 10.97, Min: 10.97  
Episode: 453, Score: 9.37, Max: 9.37, Min: 9.37   
Episode: 454, Score: 6.76, Max: 6.76, Min: 6.76   
Episode: 455, Score: 10.16, Max: 10.16, Min: 10.16   
Episode: 456, Score: 5.96, Max: 5.96, Min: 5.96  
Episode: 457, Score: 9.72, Max: 9.72, Min: 9.72  
Episode: 458, Score: 6.64, Max: 6.64, Min: 6.64  
Episode: 459, Score: 9.99, Max: 9.99, Min: 9.99  
Episode: 460, Score: 8.00, Max: 8.00, Min: 8.00  
*** Episode 460	Average Score: 7.13, Time: 01:14:56 ***  
Episode: 461, Score: 3.24, Max: 3.24, Min: 3.24  
Episode: 462, Score: 11.98, Max: 11.98, Min: 11.98  
Episode: 463, Score: 5.50, Max: 5.50, Min: 5.50  
Episode: 464, Score: 3.18, Max: 3.18, Min: 3.18  
Episode: 465, Score: 10.07, Max: 10.07, Min: 10.07  
Episode: 466, Score: 8.30, Max: 8.30, Min: 8.30  
Episode: 467, Score: 8.52, Max: 8.52, Min: 8.52  
Episode: 468, Score: 6.30, Max: 6.30, Min: 6.30  
Episode: 469, Score: 12.80, Max: 12.80, Min: 12.80  
Episode: 470, Score: 9.38, Max: 9.38, Min: 9.38  
*** Episode 470	Average Score: 7.17, Time: 01:16:56 ***  
Episode: 471, Score: 9.30, Max: 9.30, Min: 9.30  
Episode: 472, Score: 7.94, Max: 7.94, Min: 7.94  
Episode: 473, Score: 2.66, Max: 2.66, Min: 2.66  
Episode: 474, Score: 2.36, Max: 2.36, Min: 2.36  
Episode: 475, Score: 1.29, Max: 1.29, Min: 1.29  
Episode: 476, Score: 3.45, Max: 3.45, Min: 3.45  
Episode: 477, Score: 9.11, Max: 9.11, Min: 9.11  
Episode: 478, Score: 10.96, Max: 10.96, Min: 10.96  
Episode: 479, Score: 5.19, Max: 5.19, Min: 5.19  
Episode: 480, Score: 9.61, Max: 9.61, Min: 9.61  
*** Episode 480	Average Score: 6.99, Time: 01:18:56 ***  
Episode: 481, Score: 8.10, Max: 8.10, Min: 8.10  
Episode: 482, Score: 4.55, Max: 4.55, Min: 4.55  
Episode: 483, Score: 11.52, Max: 11.52, Min: 11.52  
Episode: 484, Score: 12.30, Max: 12.30, Min: 12.30  
Episode: 485, Score: 2.67, Max: 2.67, Min: 2.67  
Episode: 486, Score: 5.73, Max: 5.73, Min: 5.73  
Episode: 487, Score: 1.63, Max: 1.63, Min: 1.63  
Episode: 488, Score: 12.99, Max: 12.99, Min: 12.99  
Episode: 489, Score: 5.88, Max: 5.88, Min: 5.88  
Episode: 490, Score: 3.49, Max: 3.49, Min: 3.49  
*** Episode 490	Average Score: 6.97, Time: 01:20:58 ***  
Episode: 491, Score: 14.31, Max: 14.31, Min: 14.31  
Episode: 492, Score: 14.88, Max: 14.88, Min: 14.88  
Episode: 493, Score: 3.67, Max: 3.67, Min: 3.67  
Episode: 494, Score: 11.84, Max: 11.84, Min: 11.84  
Episode: 495, Score: 3.15, Max: 3.15, Min: 3.15  
Episode: 496, Score: 2.07, Max: 2.07, Min: 2.07  
Episode: 497, Score: 14.97, Max: 14.97, Min: 14.97  
Episode: 498, Score: 12.51, Max: 12.51, Min: 12.51  
Episode: 499, Score: 3.17, Max: 3.17, Min: 3.17  
Episode: 500, Score: 9.75, Max: 9.75, Min: 9.75  
*** Episode 500	Average Score: 7.42, Time: 01:23:00 ***  
Episode: 501, Score: 4.70, Max: 4.70, Min: 4.70  
Episode: 502, Score: 3.89, Max: 3.89, Min: 3.89  
Episode: 503, Score: 11.71, Max: 11.71, Min: 11.71  
Episode: 504, Score: 10.57, Max: 10.57, Min: 10.57  
Episode: 505, Score: 11.35, Max: 11.35, Min: 11.35  
Episode: 506, Score: 11.77, Max: 11.77, Min: 11.77  
Episode: 507, Score: 2.35, Max: 2.35, Min: 2.35  
Episode: 508, Score: 5.73, Max: 5.73, Min: 5.73  
Episode: 509, Score: 13.56, Max: 13.56, Min: 13.56  
Episode: 510, Score: 5.37, Max: 5.37, Min: 5.37  
*** Episode 510	Average Score: 7.50, Time: 01:25:04 ***  
Episode: 511, Score: 14.46, Max: 14.46, Min: 14.46  
Episode: 512, Score: 13.98, Max: 13.98, Min: 13.98  
Episode: 513, Score: 25.16, Max: 25.16, Min: 25.16  
Episode: 514, Score: 17.26, Max: 17.26, Min: 17.26  
Episode: 515, Score: 9.63, Max: 9.63, Min: 9.63  
Episode: 516, Score: 10.08, Max: 10.08, Min: 10.08  
Episode: 517, Score: 2.43, Max: 2.43, Min: 2.43  
Episode: 518, Score: 7.03, Max: 7.03, Min: 7.03  
Episode: 519, Score: 14.54, Max: 14.54, Min: 14.54  
Episode: 520, Score: 4.04, Max: 4.04, Min: 4.04  
*** Episode 520	Average Score: 8.07, Time: 01:27:09 ***  
Episode: 521, Score: 3.24, Max: 3.24, Min: 3.24  
Episode: 522, Score: 9.01, Max: 9.01, Min: 9.01  
Episode: 523, Score: 17.50, Max: 17.50, Min: 17.50  
Episode: 524, Score: 4.08, Max: 4.08, Min: 4.08  
Episode: 525, Score: 13.84, Max: 13.84, Min: 13.84  
Episode: 526, Score: 0.58, Max: 0.58, Min: 0.58  
Episode: 527, Score: 12.12, Max: 12.12, Min: 12.12  
Episode: 528, Score: 13.59, Max: 13.59, Min: 13.59  
Episode: 529, Score: 26.76, Max: 26.76, Min: 26.76  
Episode: 530, Score: 8.53, Max: 8.53, Min: 8.53  
*** Episode 530	Average Score: 8.44, Time: 01:29:15 ***  
Episode: 531, Score: 5.92, Max: 5.92, Min: 5.92  
Episode: 532, Score: 15.61, Max: 15.61, Min: 15.61  
Episode: 533, Score: 4.88, Max: 4.88, Min: 4.88  
Episode: 534, Score: 9.01, Max: 9.01, Min: 9.01  
Episode: 535, Score: 8.13, Max: 8.13, Min: 8.13  
Episode: 536, Score: 14.31, Max: 14.31, Min: 14.31  
Episode: 537, Score: 4.37, Max: 4.37, Min: 4.37  
Episode: 538, Score: 18.28, Max: 18.28, Min: 18.28  
Episode: 539, Score: 19.74, Max: 19.74, Min: 19.74  
Episode: 540, Score: 1.29, Max: 1.29, Min: 1.29  
*** Episode 540	Average Score: 8.77, Time: 01:31:23 ***  
Episode: 541, Score: 7.63, Max: 7.63, Min: 7.63  
Episode: 542, Score: 8.78, Max: 8.78, Min: 8.78  
Episode: 543, Score: 6.41, Max: 6.41, Min: 6.41  
Episode: 544, Score: 2.82, Max: 2.82, Min: 2.82  
Episode: 545, Score: 2.67, Max: 2.67, Min: 2.67  
Episode: 546, Score: 23.64, Max: 23.64, Min: 23.64  
Episode: 547, Score: 15.15, Max: 15.15, Min: 15.15  
Episode: 548, Score: 13.56, Max: 13.56, Min: 13.56  
Episode: 549, Score: 2.99, Max: 2.99, Min: 2.99  
Episode: 550, Score: 20.15, Max: 20.15, Min: 20.15  
*** Episode 550	Average Score: 9.02, Time: 01:33:32 ***   
Episode: 551, Score: 5.38, Max: 5.38, Min: 5.38   
Episode: 552, Score: 2.86, Max: 2.86, Min: 2.86   
Episode: 553, Score: 5.92, Max: 5.92, Min: 5.92   
Episode: 554, Score: 14.48, Max: 14.48, Min: 14.48   
Episode: 555, Score: 12.40, Max: 12.40, Min: 12.40   
Episode: 556, Score: 9.86, Max: 9.86, Min: 9.86   
Episode: 557, Score: 8.98, Max: 8.98, Min: 8.98   
Episode: 558, Score: 6.94, Max: 6.94, Min: 6.94    
Episode: 559, Score: 8.95, Max: 8.95, Min: 8.95  
Episode: 560, Score: 4.52, Max: 4.52, Min: 4.52  
*** Episode 560	Average Score: 8.95, Time: 01:35:42 ***  
Episode: 561, Score: 2.28, Max: 2.28, Min: 2.28  
Episode: 562, Score: 13.28, Max: 13.28, Min: 13.28  
Episode: 563, Score: 9.08, Max: 9.08, Min: 9.08  
Episode: 564, Score: 8.25, Max: 8.25, Min: 8.25  
Episode: 565, Score: 18.70, Max: 18.70, Min: 18.70  
Episode: 566, Score: 10.84, Max: 10.84, Min: 10.84  
Episode: 567, Score: 4.56, Max: 4.56, Min: 4.56  
Episode: 568, Score: 19.83, Max: 19.83, Min: 19.83  
Episode: 569, Score: 3.76, Max: 3.76, Min: 3.76  
Episode: 570, Score: 10.30, Max: 10.30, Min: 10.30  
*** Episode 570	Average Score: 9.16, Time: 01:37:54 ***  
Episode: 571, Score: 16.07, Max: 16.07, Min: 16.07  
Episode: 572, Score: 4.35, Max: 4.35, Min: 4.35  
Episode: 573, Score: 13.58, Max: 13.58, Min: 13.58  
Episode: 574, Score: 21.42, Max: 21.42, Min: 21.42  
Episode: 575, Score: 11.31, Max: 11.31, Min: 11.31  
Episode: 576, Score: 6.41, Max: 6.41, Min: 6.41   
Episode: 577, Score: 3.53, Max: 3.53, Min: 3.53  
Episode: 578, Score: 11.73, Max: 11.73, Min: 11.73  
Episode: 579, Score: 4.40, Max: 4.40, Min: 4.40  
Episode: 580, Score: 15.57, Max: 15.57, Min: 15.57  
*** Episode 580	Average Score: 9.63, Time: 01:40:07 ***  
Episode: 581, Score: 18.07, Max: 18.07, Min: 18.07  
Episode: 582, Score: 10.19, Max: 10.19, Min: 10.19  
Episode: 583, Score: 1.39, Max: 1.39, Min: 1.39  
Episode: 584, Score: 7.39, Max: 7.39, Min: 7.39  
Episode: 585, Score: 12.23, Max: 12.23, Min: 12.23  
Episode: 586, Score: 14.21, Max: 14.21, Min: 14.21  
Episode: 587, Score: 14.66, Max: 14.66, Min: 14.66  
Episode: 588, Score: 19.89, Max: 19.89, Min: 19.89  
Episode: 589, Score: 2.22, Max: 2.22, Min: 2.22  
Episode: 590, Score: 6.14, Max: 6.14, Min: 6.14  
*** Episode 590	Average Score: 10.00, Time: 01:42:21 ***  
Episode: 591, Score: 7.38, Max: 7.38, Min: 7.38  
Episode: 592, Score: 7.38, Max: 7.38, Min: 7.38  
Episode: 593, Score: 6.31, Max: 6.31, Min: 6.31  
Episode: 594, Score: 23.60, Max: 23.60, Min: 23.60  
Episode: 595, Score: 9.36, Max: 9.36, Min: 9.36  
Episode: 596, Score: 3.05, Max: 3.05, Min: 3.05  
Episode: 597, Score: 4.54, Max: 4.54, Min: 4.54  
Episode: 598, Score: 7.91, Max: 7.91, Min: 7.91  
Episode: 599, Score: 2.41, Max: 2.41, Min: 2.41  
Episode: 600, Score: 8.56, Max: 8.56, Min: 8.56  
*** Episode 600	Average Score: 9.91, Time: 01:44:37 ***  
Episode: 601, Score: 19.58, Max: 19.58, Min: 19.58  
Episode: 602, Score: 6.28, Max: 6.28, Min: 6.28  
Episode: 603, Score: 12.18, Max: 12.18, Min: 12.18  
Episode: 604, Score: 7.73, Max: 7.73, Min: 7.73  
Episode: 605, Score: 24.18, Max: 24.18, Min: 24.18  
Episode: 606, Score: 13.90, Max: 13.90, Min: 13.90  
Episode: 607, Score: 12.98, Max: 12.98, Min: 12.98  
Episode: 608, Score: 7.25, Max: 7.25, Min: 7.25  
Episode: 609, Score: 7.08, Max: 7.08, Min: 7.08  
Episode: 610, Score: 15.74, Max: 15.74, Min: 15.74  
*** Episode 610	Average Score: 10.37, Time: 01:46:53 ***  
Episode: 611, Score: 10.02, Max: 10.02, Min: 10.02  
Episode: 612, Score: 9.87, Max: 9.87, Min: 9.87  
Episode: 613, Score: 10.94, Max: 10.94, Min: 10.94  
Episode: 614, Score: 3.24, Max: 3.24, Min: 3.24  
Episode: 615, Score: 20.06, Max: 20.06, Min: 20.06  
Episode: 616, Score: 13.75, Max: 13.75, Min: 13.75  
Episode: 617, Score: 30.16, Max: 30.16, Min: 30.16  
Episode: 618, Score: 5.12, Max: 5.12, Min: 5.12  
Episode: 619, Score: 17.66, Max: 17.66, Min: 17.66  
Episode: 620, Score: 18.45, Max: 18.45, Min: 18.45  
*** Episode 620	Average Score: 10.57, Time: 01:49:11 ***  
Episode: 621, Score: 2.47, Max: 2.47, Min: 2.47  
Episode: 622, Score: 4.31, Max: 4.31, Min: 4.31  
Episode: 623, Score: 5.03, Max: 5.03, Min: 5.03  
Episode: 624, Score: 4.60, Max: 4.60, Min: 4.60  
Episode: 625, Score: 10.53, Max: 10.53, Min: 10.53  
Episode: 626, Score: 3.87, Max: 3.87, Min: 3.87  
Episode: 627, Score: 7.46, Max: 7.46, Min: 7.46  
Episode: 628, Score: 3.15, Max: 3.15, Min: 3.15  
Episode: 629, Score: 21.92, Max: 21.92, Min: 21.92  
Episode: 630, Score: 7.24, Max: 7.24, Min: 7.24  
*** Episode 630	Average Score: 10.19, Time: 01:51:30 ***  
Episode: 631, Score: 22.30, Max: 22.30, Min: 22.30  
Episode: 632, Score: 13.36, Max: 13.36, Min: 13.36  
Episode: 633, Score: 3.22, Max: 3.22, Min: 3.22  
Episode: 634, Score: 32.63, Max: 32.63, Min: 32.63  
Episode: 635, Score: 2.82, Max: 2.82, Min: 2.82  
Episode: 636, Score: 25.10, Max: 25.10, Min: 25.10  
Episode: 637, Score: 6.18, Max: 6.18, Min: 6.18  
Episode: 638, Score: 9.15, Max: 9.15, Min: 9.15  
Episode: 639, Score: 9.19, Max: 9.19, Min: 9.19  
Episode: 640, Score: 2.14, Max: 2.14, Min: 2.14  
*** Episode 640	Average Score: 10.43, Time: 01:53:51 ***  
Episode: 641, Score: 13.48, Max: 13.48, Min: 13.48  
Episode: 642, Score: 5.31, Max: 5.31, Min: 5.31  
Episode: 643, Score: 6.78, Max: 6.78, Min: 6.78  
Episode: 644, Score: 3.51, Max: 3.51, Min: 3.51  
Episode: 645, Score: 16.70, Max: 16.70, Min: 16.70  
Episode: 646, Score: 4.66, Max: 4.66, Min: 4.66  
Episode: 647, Score: 4.62, Max: 4.62, Min: 4.62  
Episode: 648, Score: 7.69, Max: 7.69, Min: 7.69  
Episode: 649, Score: 10.12, Max: 10.12, Min: 10.12  
Episode: 650, Score: 3.17, Max: 3.17, Min: 3.17  
*** Episode 650	Average Score: 10.15, Time: 01:56:13 ***  
Episode: 651, Score: 7.52, Max: 7.52, Min: 7.52  
Episode: 652, Score: 23.84, Max: 23.84, Min: 23.84  
Episode: 653, Score: 25.11, Max: 25.11, Min: 25.11  
Episode: 654, Score: 13.83, Max: 13.83, Min: 13.83  
Episode: 655, Score: 19.08, Max: 19.08, Min: 19.08  
Episode: 656, Score: 4.44, Max: 4.44, Min: 4.44  
Episode: 657, Score: 16.01, Max: 16.01, Min: 16.01  
Episode: 658, Score: 8.57, Max: 8.57, Min: 8.57  
Episode: 659, Score: 6.87, Max: 6.87, Min: 6.87  
Episode: 660, Score: 19.06, Max: 19.06, Min: 19.06  
*** Episode 660	Average Score: 10.79, Time: 01:58:37 ***  
Episode: 661, Score: 2.87, Max: 2.87, Min: 2.87  
Episode: 662, Score: 2.06, Max: 2.06, Min: 2.06  
Episode: 663, Score: 28.06, Max: 28.06, Min: 28.06  
Episode: 664, Score: 11.78, Max: 11.78, Min: 11.78  
Episode: 665, Score: 19.31, Max: 19.31, Min: 19.31  
Episode: 666, Score: 5.32, Max: 5.32, Min: 5.32  
Episode: 667, Score: 17.88, Max: 17.88, Min: 17.88  
Episode: 668, Score: 16.40, Max: 16.40, Min: 16.40   
Episode: 669, Score: 18.50, Max: 18.50, Min: 18.50  
Episode: 670, Score: 3.20, Max: 3.20, Min: 3.20  
*** Episode 670	Average Score: 11.04, Time: 02:01:02 ***  
Episode: 671, Score: 3.20, Max: 3.20, Min: 3.20  
Episode: 672, Score: 15.57, Max: 15.57, Min: 15.57  
Episode: 673, Score: 15.78, Max: 15.78, Min: 15.78  
Episode: 674, Score: 9.36, Max: 9.36, Min: 9.36  
Episode: 675, Score: 18.67, Max: 18.67, Min: 18.67  
Episode: 676, Score: 2.92, Max: 2.92, Min: 2.92  
Episode: 677, Score: 19.20, Max: 19.20, Min: 19.20  
Episode: 678, Score: 16.25, Max: 16.25, Min: 16.25  
Episode: 679, Score: 2.01, Max: 2.01, Min: 2.01  
Episode: 680, Score: 22.43, Max: 22.43, Min: 22.43  
*** Episode 680	Average Score: 11.21, Time: 02:03:31 ***  
Episode: 681, Score: 10.06, Max: 10.06, Min: 10.06  
Episode: 682, Score: 4.09, Max: 4.09, Min: 4.09  
Episode: 683, Score: 19.46, Max: 19.46, Min: 19.46  
Episode: 684, Score: 17.95, Max: 17.95, Min: 17.95  
Episode: 685, Score: 19.75, Max: 19.75, Min: 19.75  
Episode: 686, Score: 9.20, Max: 9.20, Min: 9.20  
Episode: 687, Score: 11.66, Max: 11.66, Min: 11.66  
Episode: 688, Score: 8.31, Max: 8.31, Min: 8.31  
Episode: 689, Score: 3.53, Max: 3.53, Min: 3.53  
Episode: 690, Score: 2.82, Max: 2.82, Min: 2.82  
*** Episode 690	Average Score: 11.21, Time: 02:06:01 ***  
Episode: 691, Score: 19.89, Max: 19.89, Min: 19.89  
Episode: 692, Score: 13.58, Max: 13.58, Min: 13.58  
Episode: 693, Score: 2.47, Max: 2.47, Min: 2.47   
Episode: 694, Score: 29.85, Max: 29.85, Min: 29.85  
Episode: 695, Score: 15.01, Max: 15.01, Min: 15.01  
Episode: 696, Score: 2.25, Max: 2.25, Min: 2.25  
Episode: 697, Score: 2.07, Max: 2.07, Min: 2.07  
Episode: 698, Score: 14.41, Max: 14.41, Min: 14.41  
Episode: 699, Score: 24.73, Max: 24.73, Min: 24.73  
Episode: 700, Score: 19.38, Max: 19.38, Min: 19.38  
*** Episode 700	Average Score: 11.84, Time: 02:08:31 ***  
Episode: 701, Score: 13.68, Max: 13.68, Min: 13.68  
Episode: 702, Score: 21.52, Max: 21.52, Min: 21.52  
Episode: 703, Score: 17.20, Max: 17.20, Min: 17.20  
Episode: 704, Score: 2.43, Max: 2.43, Min: 2.43  
Episode: 705, Score: 3.79, Max: 3.79, Min: 3.79  
Episode: 706, Score: 13.54, Max: 13.54, Min: 13.54  
Episode: 707, Score: 6.04, Max: 6.04, Min: 6.04  
Episode: 708, Score: 14.38, Max: 14.38, Min: 14.38  
Episode: 709, Score: 8.24, Max: 8.24, Min: 8.24  
Episode: 710, Score: 29.28, Max: 29.28, Min: 29.28  
*** Episode 710	Average Score: 11.88, Time: 02:11:03 ***  
Episode: 711, Score: 18.07, Max: 18.07, Min: 18.07  
Episode: 712, Score: 27.06, Max: 27.06, Min: 27.06  
Episode: 713, Score: 8.72, Max: 8.72, Min: 8.72  
Episode: 714, Score: 3.57, Max: 3.57, Min: 3.57  
Episode: 715, Score: 6.51, Max: 6.51, Min: 6.51  
Episode: 716, Score: 3.81, Max: 3.81, Min: 3.81  
Episode: 717, Score: 4.26, Max: 4.26, Min: 4.26  
Episode: 718, Score: 21.37, Max: 21.37, Min: 21.37  
Episode: 719, Score: 5.45, Max: 5.45, Min: 5.45  
Episode: 720, Score: 19.41, Max: 19.41, Min: 19.41  
*** Episode 720	Average Score: 11.67, Time: 02:13:36 ***  
Episode: 721, Score: 17.35, Max: 17.35, Min: 17.35  
Episode: 722, Score: 4.12, Max: 4.12, Min: 4.12  
Episode: 723, Score: 3.38, Max: 3.38, Min: 3.38  
Episode: 724, Score: 22.65, Max: 22.65, Min: 22.65  
Episode: 725, Score: 8.54, Max: 8.54, Min: 8.54  
Episode: 726, Score: 26.04, Max: 26.04, Min: 26.04  
Episode: 727, Score: 2.54, Max: 2.54, Min: 2.54  
Episode: 728, Score: 1.87, Max: 1.87, Min: 1.87  
Episode: 729, Score: 17.88, Max: 17.88, Min: 17.88  
Episode: 730, Score: 15.07, Max: 15.07, Min: 15.07  
*** Episode 730	Average Score: 12.15, Time: 02:16:12 ***  
Episode: 731, Score: 7.21, Max: 7.21, Min: 7.21  
Episode: 732, Score: 5.27, Max: 5.27, Min: 5.27  
Episode: 733, Score: 14.66, Max: 14.66, Min: 14.66  
Episode: 734, Score: 16.94, Max: 16.94, Min: 16.94  
Episode: 735, Score: 2.42, Max: 2.42, Min: 2.42  
Episode: 736, Score: 6.22, Max: 6.22, Min: 6.22  
Episode: 737, Score: 15.59, Max: 15.59, Min: 15.59  
Episode: 738, Score: 20.03, Max: 20.03, Min: 20.03  
Episode: 739, Score: 13.51, Max: 13.51, Min: 13.51  
Episode: 740, Score: 26.92, Max: 26.92, Min: 26.92  
*** Episode 740	Average Score: 12.18, Time: 02:18:47 ***  
Episode: 741, Score: 19.77, Max: 19.77, Min: 19.77  
Episode: 742, Score: 18.89, Max: 18.89, Min: 18.89  
Episode: 743, Score: 26.02, Max: 26.02, Min: 26.02  
Episode: 744, Score: 13.55, Max: 13.55, Min: 13.55  
Episode: 745, Score: 24.26, Max: 24.26, Min: 24.26  
Episode: 746, Score: 25.13, Max: 25.13, Min: 25.13  
Episode: 747, Score: 29.67, Max: 29.67, Min: 29.67  
Episode: 748, Score: 4.67, Max: 4.67, Min: 4.67  
Episode: 749, Score: 20.35, Max: 20.35, Min: 20.35  
Episode: 750, Score: 19.79, Max: 19.79, Min: 19.79  
*** Episode 750	Average Score: 13.44, Time: 02:21:24 ***  
Episode: 751, Score: 7.03, Max: 7.03, Min: 7.03  
Episode: 752, Score: 16.32, Max: 16.32, Min: 16.32  
Episode: 753, Score: 26.17, Max: 26.17, Min: 26.17   
Episode: 754, Score: 32.53, Max: 32.53, Min: 32.53  
Episode: 755, Score: 13.32, Max: 13.32, Min: 13.32  
Episode: 756, Score: 27.52, Max: 27.52, Min: 27.52  
Episode: 757, Score: 19.58, Max: 19.58, Min: 19.58  
Episode: 758, Score: 32.15, Max: 32.15, Min: 32.15  
Episode: 759, Score: 4.51, Max: 4.51, Min: 4.51  
Episode: 760, Score: 7.58, Max: 7.58, Min: 7.58  
*** Episode 760	Average Score: 13.87, Time: 02:24:03 ***  
Episode: 761, Score: 3.85, Max: 3.85, Min: 3.85  
Episode: 762, Score: 28.24, Max: 28.24, Min: 28.24  
Episode: 763, Score: 29.66, Max: 29.66, Min: 29.66  
Episode: 764, Score: 35.16, Max: 35.16, Min: 35.16  
Episode: 765, Score: 10.78, Max: 10.78, Min: 10.78  
Episode: 766, Score: 2.93, Max: 2.93, Min: 2.93  
Episode: 767, Score: 23.83, Max: 23.83, Min: 23.83  
Episode: 768, Score: 10.01, Max: 10.01, Min: 10.01  
Episode: 769, Score: 18.32, Max: 18.32, Min: 18.32  
Episode: 770, Score: 19.30, Max: 19.30, Min: 19.30  
*** Episode 770	Average Score: 14.43, Time: 02:26:41 ***  
Episode: 771, Score: 19.44, Max: 19.44, Min: 19.44  
Episode: 772, Score: 8.00, Max: 8.00, Min: 8.00  
Episode: 773, Score: 21.02, Max: 21.02, Min: 21.02  
Episode: 774, Score: 26.65, Max: 26.65, Min: 26.65  
Episode: 775, Score: 20.87, Max: 20.87, Min: 20.87  
Episode: 776, Score: 2.38, Max: 2.38, Min: 2.38  
Episode: 777, Score: 11.83, Max: 11.83, Min: 11.83  
Episode: 778, Score: 28.79, Max: 28.79, Min: 28.79  
Episode: 779, Score: 6.53, Max: 6.53, Min: 6.53  
Episode: 780, Score: 24.57, Max: 24.57, Min: 24.57  
*** Episode 780	Average Score: 14.88, Time: 02:29:20 ***  
Episode: 781, Score: 8.01, Max: 8.01, Min: 8.01  
Episode: 782, Score: 24.77, Max: 24.77, Min: 24.77  
Episode: 783, Score: 6.88, Max: 6.88, Min: 6.88   
Episode: 784, Score: 21.71, Max: 21.71, Min: 21.71   
Episode: 785, Score: 38.60, Max: 38.60, Min: 38.60   
Episode: 786, Score: 16.05, Max: 16.05, Min: 16.05  
Episode: 787, Score: 15.08, Max: 15.08, Min: 15.08  
Episode: 788, Score: 31.80, Max: 31.80, Min: 31.80  
Episode: 789, Score: 16.49, Max: 16.49, Min: 16.49  
Episode: 790, Score: 27.48, Max: 27.48, Min: 27.48  
*** Episode 790	Average Score: 15.88, Time: 02:32:03 ***  
Episode: 791, Score: 15.82, Max: 15.82, Min: 15.82  
Episode: 792, Score: 16.33, Max: 16.33, Min: 16.33  
Episode: 793, Score: 2.53, Max: 2.53, Min: 2.53  
Episode: 794, Score: 32.53, Max: 32.53, Min: 32.53  
Episode: 795, Score: 23.47, Max: 23.47, Min: 23.47  
Episode: 796, Score: 2.60, Max: 2.60, Min: 2.60  
Episode: 797, Score: 21.56, Max: 21.56, Min: 21.56  
Episode: 798, Score: 21.04, Max: 21.04, Min: 21.04  
Episode: 799, Score: 2.07, Max: 2.07, Min: 2.07  
Episode: 800, Score: 19.31, Max: 19.31, Min: 19.31  
*** Episode 800	Average Score: 16.02, Time: 02:34:47 ***  
Episode: 801, Score: 3.49, Max: 3.49, Min: 3.49  
Episode: 802, Score: 2.22, Max: 2.22, Min: 2.22  
Episode: 803, Score: 28.21, Max: 28.21, Min: 28.21  
Episode: 804, Score: 21.68, Max: 21.68, Min: 21.68  
Episode: 805, Score: 16.95, Max: 16.95, Min: 16.95  
Episode: 806, Score: 25.84, Max: 25.84, Min: 25.84  
Episode: 807, Score: 24.47, Max: 24.47, Min: 24.47  
Episode: 808, Score: 2.95, Max: 2.95, Min: 2.95  
Episode: 809, Score: 3.09, Max: 3.09, Min: 3.09  
Episode: 810, Score: 29.64, Max: 29.64, Min: 29.64  
*** Episode 810	Average Score: 16.30, Time: 02:37:31 ***  
Episode: 811, Score: 1.46, Max: 1.46, Min: 1.46  
Episode: 812, Score: 5.71, Max: 5.71, Min: 5.71  
Episode: 813, Score: 26.14, Max: 26.14, Min: 26.14   
Episode: 814, Score: 20.25, Max: 20.25, Min: 20.25  
Episode: 815, Score: 7.11, Max: 7.11, Min: 7.11  
Episode: 816, Score: 24.23, Max: 24.23, Min: 24.23  
Episode: 817, Score: 30.69, Max: 30.69, Min: 30.69  
Episode: 818, Score: 3.74, Max: 3.74, Min: 3.74  
Episode: 819, Score: 2.41, Max: 2.41, Min: 2.41  
Episode: 820, Score: 21.64, Max: 21.64, Min: 21.64  
*** Episode 820	Average Score: 16.55, Time: 02:40:16 ***  
Episode: 821, Score: 1.48, Max: 1.48, Min: 1.48  
Episode: 822, Score: 14.53, Max: 14.53, Min: 14.53  
Episode: 823, Score: 29.08, Max: 29.08, Min: 29.08  
Episode: 824, Score: 1.85, Max: 1.85, Min: 1.85  
Episode: 825, Score: 2.11, Max: 2.11, Min: 2.11  
Episode: 826, Score: 2.05, Max: 2.05, Min: 2.05  
Episode: 827, Score: 18.76, Max: 18.76, Min: 18.76  
Episode: 828, Score: 26.29, Max: 26.29, Min: 26.29  
Episode: 829, Score: 15.82, Max: 15.82, Min: 15.82  
Episode: 830, Score: 22.66, Max: 22.66, Min: 22.66  
*** Episode 830	Average Score: 16.70, Time: 02:43:04 ***  
Episode: 831, Score: 20.78, Max: 20.78, Min: 20.78  
Episode: 832, Score: 2.74, Max: 2.74, Min: 2.74   
Episode: 833, Score: 29.79, Max: 29.79, Min: 29.79  
Episode: 834, Score: 21.79, Max: 21.79, Min: 21.79   
Episode: 835, Score: 24.73, Max: 24.73, Min: 24.73   
Episode: 836, Score: 22.42, Max: 22.42, Min: 22.42  
Episode: 837, Score: 26.38, Max: 26.38, Min: 26.38  
Episode: 838, Score: 25.45, Max: 25.45, Min: 25.45  
Episode: 839, Score: 30.08, Max: 30.08, Min: 30.08   
Episode: 840, Score: 14.02, Max: 14.02, Min: 14.02    
*** Episode 840	Average Score: 17.60, Time: 02:45:52 ***  
Episode: 841, Score: 27.82, Max: 27.82, Min: 27.82  
Episode: 842, Score: 21.93, Max: 21.93, Min: 21.93  
Episode: 843, Score: 28.87, Max: 28.87, Min: 28.87  
Episode: 844, Score: 8.94, Max: 8.94, Min: 8.94  
Episode: 845, Score: 28.00, Max: 28.00, Min: 28.00  
Episode: 846, Score: 28.95, Max: 28.95, Min: 28.95  
Episode: 847, Score: 24.24, Max: 24.24, Min: 24.24  
Episode: 848, Score: 24.88, Max: 24.88, Min: 24.88  
Episode: 849, Score: 25.42, Max: 25.42, Min: 25.42  
Episode: 850, Score: 32.73, Max: 32.73, Min: 32.73  
*** Episode 850	Average Score: 18.10, Time: 02:48:42 ***  
Episode: 851, Score: 25.93, Max: 25.93, Min: 25.93  
Episode: 852, Score: 25.50, Max: 25.50, Min: 25.50  
Episode: 853, Score: 7.82, Max: 7.82, Min: 7.82  
Episode: 854, Score: 24.91, Max: 24.91, Min: 24.91  
Episode: 855, Score: 18.78, Max: 18.78, Min: 18.78  
Episode: 856, Score: 31.07, Max: 31.07, Min: 31.07  
Episode: 857, Score: 25.63, Max: 25.63, Min: 25.63  
Episode: 858, Score: 16.20, Max: 16.20, Min: 16.20  
Episode: 859, Score: 29.01, Max: 29.01, Min: 29.01  
Episode: 860, Score: 16.55, Max: 16.55, Min: 16.55   
*** Episode 860	Average Score: 18.44, Time: 02:51:32 ***  
Episode: 861, Score: 18.42, Max: 18.42, Min: 18.42  
Episode: 862, Score: 17.76, Max: 17.76, Min: 17.76  
Episode: 863, Score: 1.37, Max: 1.37, Min: 1.37  
Episode: 864, Score: 25.55, Max: 25.55, Min: 25.55  
Episode: 865, Score: 18.37, Max: 18.37, Min: 18.37  
Episode: 866, Score: 5.06, Max: 5.06, Min: 5.06   
Episode: 867, Score: 16.86, Max: 16.86, Min: 16.86  
Episode: 868, Score: 25.09, Max: 25.09, Min: 25.09  
Episode: 869, Score: 16.59, Max: 16.59, Min: 16.59  
Episode: 870, Score: 10.88, Max: 10.88, Min: 10.88  
*** Episode 870	Average Score: 18.18, Time: 02:54:24 ***  
Episode: 871, Score: 29.21, Max: 29.21, Min: 29.21  
Episode: 872, Score: 1.68, Max: 1.68, Min: 1.68  
Episode: 873, Score: 23.99, Max: 23.99, Min: 23.99  
Episode: 874, Score: 9.73, Max: 9.73, Min: 9.73  
Episode: 875, Score: 20.48, Max: 20.48, Min: 20.48   
Episode: 876, Score: 24.50, Max: 24.50, Min: 24.50   
Episode: 877, Score: 27.85, Max: 27.85, Min: 27.85   
Episode: 878, Score: 24.75, Max: 24.75, Min: 24.75   
Episode: 879, Score: 19.16, Max: 19.16, Min: 19.16   
Episode: 880, Score: 20.98, Max: 20.98, Min: 20.98  
*** Episode 880	Average Score: 18.50, Time: 02:57:16 ***   
Episode: 881, Score: 25.86, Max: 25.86, Min: 25.86  
Episode: 882, Score: 13.35, Max: 13.35, Min: 13.35  
Episode: 883, Score: 25.93, Max: 25.93, Min: 25.93  
Episode: 884, Score: 16.93, Max: 16.93, Min: 16.93  
Episode: 885, Score: 22.02, Max: 22.02, Min: 22.02  
Episode: 886, Score: 3.69, Max: 3.69, Min: 3.69  
Episode: 887, Score: 16.51, Max: 16.51, Min: 16.51  
Episode: 888, Score: 9.55, Max: 9.55, Min: 9.55  
Episode: 889, Score: 33.04, Max: 33.04, Min: 33.04  
Episode: 890, Score: 23.01, Max: 23.01, Min: 23.01  
*** Episode 890	Average Score: 18.33, Time: 03:00:09 ***  
Episode: 891, Score: 12.49, Max: 12.49, Min: 12.49  
Episode: 892, Score: 12.37, Max: 12.37, Min: 12.37  
Episode: 893, Score: 22.80, Max: 22.80, Min: 22.80  
Episode: 894, Score: 27.30, Max: 27.30, Min: 27.30  
Episode: 895, Score: 14.36, Max: 14.36, Min: 14.36  
Episode: 896, Score: 1.18, Max: 1.18, Min: 1.18  
Episode: 897, Score: 30.60, Max: 30.60, Min: 30.60  
Episode: 898, Score: 13.28, Max: 13.28, Min: 13.28  
Episode: 899, Score: 29.90, Max: 29.90, Min: 29.90  
Episode: 900, Score: 20.51, Max: 20.51, Min: 20.51  
*** Episode 900	Average Score: 18.61, Time: 03:03:03 ***  
Episode: 901, Score: 21.21, Max: 21.21, Min: 21.21  
Episode: 902, Score: 23.68, Max: 23.68, Min: 23.68  
Episode: 903, Score: 28.20, Max: 28.20, Min: 28.20  
Episode: 904, Score: 29.02, Max: 29.02, Min: 29.02  
Episode: 905, Score: 31.70, Max: 31.70, Min: 31.70  
Episode: 906, Score: 18.42, Max: 18.42, Min: 18.42  
Episode: 907, Score: 19.47, Max: 19.47, Min: 19.47  
Episode: 908, Score: 17.06, Max: 17.06, Min: 17.06  
Episode: 909, Score: 16.17, Max: 16.17, Min: 16.17  
Episode: 910, Score: 18.31, Max: 18.31, Min: 18.31  
*** Episode 910	Average Score: 19.26, Time: 03:06:00 ***  
Episode: 911, Score: 24.45, Max: 24.45, Min: 24.45  
Episode: 912, Score: 33.92, Max: 33.92, Min: 33.92  
Episode: 913, Score: 12.56, Max: 12.56, Min: 12.56  
Episode: 914, Score: 21.99, Max: 21.99, Min: 21.99  
Episode: 915, Score: 18.94, Max: 18.94, Min: 18.94  
Episode: 916, Score: 23.80, Max: 23.80, Min: 23.80  
Episode: 917, Score: 26.83, Max: 26.83, Min: 26.83  
Episode: 918, Score: 31.95, Max: 31.95, Min: 31.95  
Episode: 919, Score: 22.05, Max: 22.05, Min: 22.05  
Episode: 920, Score: 29.66, Max: 29.66, Min: 29.66  
*** Episode 920	Average Score: 20.28, Time: 03:09:01 ***  
Episode: 921, Score: 16.31, Max: 16.31, Min: 16.31  
Episode: 922, Score: 16.34, Max: 16.34, Min: 16.34  
Episode: 923, Score: 20.64, Max: 20.64, Min: 20.64  
Episode: 924, Score: 26.50, Max: 26.50, Min: 26.50  
Episode: 925, Score: 30.69, Max: 30.69, Min: 30.69  
Episode: 926, Score: 14.83, Max: 14.83, Min: 14.83  
Episode: 927, Score: 7.38, Max: 7.38, Min: 7.38  
Episode: 928, Score: 13.64, Max: 13.64, Min: 13.64  
Episode: 929, Score: 27.61, Max: 27.61, Min: 27.61  
Episode: 930, Score: 7.94, Max: 7.94, Min: 7.94  
*** Episode 930	Average Score: 20.76, Time: 03:12:00 ***  
Episode: 931, Score: 7.06, Max: 7.06, Min: 7.06  
Episode: 932, Score: 15.31, Max: 15.31, Min: 15.31  
Episode: 933, Score: 38.86, Max: 38.86, Min: 38.86  
Episode: 934, Score: 15.51, Max: 15.51, Min: 15.51  
Episode: 935, Score: 24.93, Max: 24.93, Min: 24.93  
Episode: 936, Score: 11.98, Max: 11.98, Min: 11.98  
Episode: 937, Score: 18.07, Max: 18.07, Min: 18.07  
Episode: 938, Score: 12.69, Max: 12.69, Min: 12.69  
Episode: 939, Score: 5.46, Max: 5.46, Min: 5.46  
Episode: 940, Score: 3.39, Max: 3.39, Min: 3.39  
*** Episode 940	Average Score: 20.11, Time: 03:15:00 ***  
Episode: 941, Score: 29.31, Max: 29.31, Min: 29.31  
Episode: 942, Score: 2.59, Max: 2.59, Min: 2.59  
Episode: 943, Score: 16.66, Max: 16.66, Min: 16.66  
Episode: 944, Score: 29.68, Max: 29.68, Min: 29.68  
Episode: 945, Score: 32.27, Max: 32.27, Min: 32.27  
Episode: 946, Score: 16.18, Max: 16.18, Min: 16.18  
Episode: 947, Score: 25.33, Max: 25.33, Min: 25.33  
Episode: 948, Score: 25.25, Max: 25.25, Min: 25.25  
Episode: 949, Score: 25.96, Max: 25.96, Min: 25.96  
Episode: 950, Score: 25.45, Max: 25.45, Min: 25.45  
*** Episode 950	Average Score: 19.88, Time: 03:18:00 ***  
Episode: 951, Score: 31.15, Max: 31.15, Min: 31.15  
Episode: 952, Score: 14.39, Max: 14.39, Min: 14.39  
Episode: 953, Score: 34.82, Max: 34.82, Min: 34.82  
Episode: 954, Score: 21.05, Max: 21.05, Min: 21.05  
Episode: 955, Score: 30.50, Max: 30.50, Min: 30.50  
Episode: 956, Score: 4.16, Max: 4.16, Min: 4.16  
Episode: 957, Score: 19.31, Max: 19.31, Min: 19.31  
Episode: 958, Score: 28.40, Max: 28.40, Min: 28.40  
Episode: 959, Score: 30.75, Max: 30.75, Min: 30.75  
Episode: 960, Score: 20.06, Max: 20.06, Min: 20.06  
*** Episode 960	Average Score: 20.01, Time: 03:21:02 ***  
Episode: 961, Score: 7.36, Max: 7.36, Min: 7.36  
Episode: 962, Score: 32.89, Max: 32.89, Min: 32.89  
Episode: 963, Score: 7.09, Max: 7.09, Min: 7.09  
Episode: 964, Score: 4.30, Max: 4.30, Min: 4.30  
Episode: 965, Score: 13.20, Max: 13.20, Min: 13.20  
Episode: 966, Score: 18.21, Max: 18.21, Min: 18.21  
Episode: 967, Score: 31.05, Max: 31.05, Min: 31.05  
Episode: 968, Score: 35.27, Max: 35.27, Min: 35.27  
Episode: 969, Score: 29.06, Max: 29.06, Min: 29.06  
Episode: 970, Score: 31.01, Max: 31.01, Min: 31.01  
*** Episode 970	Average Score: 20.54, Time: 03:24:05 ***  
Episode: 971, Score: 18.13, Max: 18.13, Min: 18.13  
Episode: 972, Score: 25.28, Max: 25.28, Min: 25.28  
Episode: 973, Score: 3.81, Max: 3.81, Min: 3.81  
Episode: 974, Score: 21.33, Max: 21.33, Min: 21.33  
Episode: 975, Score: 26.89, Max: 26.89, Min: 26.89  
Episode: 976, Score: 27.94, Max: 27.94, Min: 27.94  
Episode: 977, Score: 31.58, Max: 31.58, Min: 31.58  
Episode: 978, Score: 25.23, Max: 25.23, Min: 25.23  
Episode: 979, Score: 13.38, Max: 13.38, Min: 13.38  
Episode: 980, Score: 12.83, Max: 12.83, Min: 12.83  
*** Episode 980	Average Score: 20.58, Time: 03:27:09 ***  
Episode: 981, Score: 27.67, Max: 27.67, Min: 27.67  
Episode: 982, Score: 2.00, Max: 2.00, Min: 2.00  
Episode: 983, Score: 28.87, Max: 28.87, Min: 28.87  
Episode: 984, Score: 26.01, Max: 26.01, Min: 26.01  
Episode: 985, Score: 8.75, Max: 8.75, Min: 8.75  
Episode: 986, Score: 28.98, Max: 28.98, Min: 28.98  
Episode: 987, Score: 12.87, Max: 12.87, Min: 12.87  
Episode: 988, Score: 30.48, Max: 30.48, Min: 30.48  
Episode: 989, Score: 9.04, Max: 9.04, Min: 9.04  
Episode: 990, Score: 3.87, Max: 3.87, Min: 3.87  
*** Episode 990	Average Score: 20.47, Time: 03:30:15 ***  
Episode: 991, Score: 7.39, Max: 7.39, Min: 7.39  
Episode: 992, Score: 19.39, Max: 19.39, Min: 19.39  
Episode: 993, Score: 29.04, Max: 29.04, Min: 29.04  
Episode: 994, Score: 26.58, Max: 26.58, Min: 26.58  
Episode: 995, Score: 32.20, Max: 32.20, Min: 32.20   
Episode: 996, Score: 28.96, Max: 28.96, Min: 28.96  
Episode: 997, Score: 24.05, Max: 24.05, Min: 24.05   
Episode: 998, Score: 22.50, Max: 22.50, Min: 22.50  
Episode: 999, Score: 28.04, Max: 28.04, Min: 28.04  
Episode: 1000, Score: 35.07, Max: 35.07, Min: 35.07  
*** Episode 1000	Average Score: 21.15, Time: 03:33:22 ***   
Episode: 1001, Score: 15.01, Max: 15.01, Min: 15.01  
Episode: 1002, Score: 11.63, Max: 11.63, Min: 11.63  
Episode: 1003, Score: 33.27, Max: 33.27, Min: 33.27  
Episode: 1004, Score: 27.32, Max: 27.32, Min: 27.32  
Episode: 1005, Score: 6.55, Max: 6.55, Min: 6.55   
Episode: 1006, Score: 34.63, Max: 34.63, Min: 34.63  
Episode: 1007, Score: 29.58, Max: 29.58, Min: 29.58  
Episode: 1008, Score: 26.09, Max: 26.09, Min: 26.09  
Episode: 1009, Score: 26.41, Max: 26.41, Min: 26.41  
Episode: 1010, Score: 33.79, Max: 33.79, Min: 33.79  
*** Episode 1010	Average Score: 21.36, Time: 03:36:30 ***  
Episode: 1011, Score: 27.98, Max: 27.98, Min: 27.98   
Episode: 1012, Score: 11.05, Max: 11.05, Min: 11.05  
Episode: 1013, Score: 33.41, Max: 33.41, Min: 33.41  
Episode: 1014, Score: 26.92, Max: 26.92, Min: 26.92  
Episode: 1015, Score: 31.29, Max: 31.29, Min: 31.29  
Episode: 1016, Score: 26.95, Max: 26.95, Min: 26.95  
Episode: 1017, Score: 28.92, Max: 28.92, Min: 28.92  
Episode: 1018, Score: 30.79, Max: 30.79, Min: 30.79  
Episode: 1019, Score: 33.88, Max: 33.88, Min: 33.88  
Episode: 1020, Score: 25.78, Max: 25.78, Min: 25.78  
*** Episode 1020	Average Score: 21.67, Time: 03:39:39 ***  
Episode: 1021, Score: 14.85, Max: 14.85, Min: 14.85  
Episode: 1022, Score: 7.72, Max: 7.72, Min: 7.72  
Episode: 1023, Score: 8.32, Max: 8.32, Min: 8.32  
Episode: 1024, Score: 1.97, Max: 1.97, Min: 1.97  
Episode: 1025, Score: 28.44, Max: 28.44, Min: 28.44   
Episode: 1026, Score: 25.68, Max: 25.68, Min: 25.68   
Episode: 1027, Score: 25.67, Max: 25.67, Min: 25.67   
Episode: 1028, Score: 25.45, Max: 25.45, Min: 25.45   
Episode: 1029, Score: 21.57, Max: 21.57, Min: 21.57   
Episode: 1030, Score: 0.50, Max: 0.50, Min: 0.50   
*** Episode 1030	Average Score: 21.46, Time: 03:42:47 ***    
Episode: 1031, Score: 28.73, Max: 28.73, Min: 28.73   
Episode: 1032, Score: 28.21, Max: 28.21, Min: 28.21  
Episode: 1033, Score: 29.06, Max: 29.06, Min: 29.06   
Episode: 1034, Score: 27.74, Max: 27.74, Min: 27.74   
Episode: 1035, Score: 34.70, Max: 34.70, Min: 34.70   
Episode: 1036, Score: 28.21, Max: 28.21, Min: 28.21   
Episode: 1037, Score: 31.35, Max: 31.35, Min: 31.35  
Episode: 1038, Score: 2.64, Max: 2.64, Min: 2.64  
Episode: 1039, Score: 32.64, Max: 32.64, Min: 32.64  
Episode: 1040, Score: 32.02, Max: 32.02, Min: 32.02  
*** Episode 1040	Average Score: 22.68, Time: 03:45:56 ***  
Episode: 1041, Score: 1.20, Max: 1.20, Min: 1.20  
Episode: 1042, Score: 28.45, Max: 28.45, Min: 28.45  
Episode: 1043, Score: 13.67, Max: 13.67, Min: 13.67  
Episode: 1044, Score: 26.26, Max: 26.26, Min: 26.26  
Episode: 1045, Score: 28.31, Max: 28.31, Min: 28.31  
Episode: 1046, Score: 33.29, Max: 33.29, Min: 33.29  
Episode: 1047, Score: 25.45, Max: 25.45, Min: 25.45  
Episode: 1048, Score: 29.80, Max: 29.80, Min: 29.80  
Episode: 1049, Score: 29.74, Max: 29.74, Min: 29.74  
Episode: 1050, Score: 28.51, Max: 28.51, Min: 28.51  
*** Episode 1050	Average Score: 22.84, Time: 03:49:04 ***  
Episode: 1051, Score: 8.60, Max: 8.60, Min: 8.60  
Episode: 1052, Score: 1.35, Max: 1.35, Min: 1.35  
Episode: 1053, Score: 36.59, Max: 36.59, Min: 36.59  
Episode: 1054, Score: 7.32, Max: 7.32, Min: 7.32  
Episode: 1055, Score: 32.15, Max: 32.15, Min: 32.15  
Episode: 1056, Score: 34.33, Max: 34.33, Min: 34.33  
Episode: 1057, Score: 29.26, Max: 29.26, Min: 29.26  
Episode: 1058, Score: 32.41, Max: 32.41, Min: 32.41  
Episode: 1059, Score: 26.21, Max: 26.21, Min: 26.21  
Episode: 1060, Score: 31.15, Max: 31.15, Min: 31.15  
*** Episode 1060	Average Score: 22.88, Time: 03:52:13 ***  
Episode: 1061, Score: 30.36, Max: 30.36, Min: 30.36  
Episode: 1062, Score: 6.23, Max: 6.23, Min: 6.23  
Episode: 1063, Score: 28.11, Max: 28.11, Min: 28.11  
Episode: 1064, Score: 23.89, Max: 23.89, Min: 23.89  
Episode: 1065, Score: 20.27, Max: 20.27, Min: 20.27  
Episode: 1066, Score: 27.08, Max: 27.08, Min: 27.08  
Episode: 1067, Score: 21.96, Max: 21.96, Min: 21.96  
Episode: 1068, Score: 29.59, Max: 29.59, Min: 29.59  
Episode: 1069, Score: 27.53, Max: 27.53, Min: 27.53  
Episode: 1070, Score: 0.97, Max: 0.97, Min: 0.97  
*** Episode 1070	Average Score: 22.95, Time: 03:55:21 ***  
Episode: 1071, Score: 9.04, Max: 9.04, Min: 9.04  
Episode: 1072, Score: 18.32, Max: 18.32, Min: 18.32  
Episode: 1073, Score: 27.89, Max: 27.89, Min: 27.89  
Episode: 1074, Score: 28.97, Max: 28.97, Min: 28.97  
Episode: 1075, Score: 33.46, Max: 33.46, Min: 33.46  
Episode: 1076, Score: 27.02, Max: 27.02, Min: 27.02  
Episode: 1077, Score: 16.36, Max: 16.36, Min: 16.36   
Episode: 1078, Score: 25.34, Max: 25.34, Min: 25.34   
Episode: 1079, Score: 24.05, Max: 24.05, Min: 24.05  
Episode: 1080, Score: 23.21, Max: 23.21, Min: 23.21  
*** Episode 1080	Average Score: 23.22, Time: 03:58:30 ***  
Episode: 1081, Score: 28.73, Max: 28.73, Min: 28.73  
Episode: 1082, Score: 2.89, Max: 2.89, Min: 2.89  
Episode: 1083, Score: 29.73, Max: 29.73, Min: 29.73  
Episode: 1084, Score: 23.82, Max: 23.82, Min: 23.82  
Episode: 1085, Score: 27.29, Max: 27.29, Min: 27.29  
Episode: 1086, Score: 30.70, Max: 30.70, Min: 30.70  
Episode: 1087, Score: 8.55, Max: 8.55, Min: 8.55  
Episode: 1088, Score: 7.48, Max: 7.48, Min: 7.48  
Episode: 1089, Score: 13.42, Max: 13.42, Min: 13.42  
Episode: 1090, Score: 28.77, Max: 28.77, Min: 28.77  
*** Episode 1090	Average Score: 23.45, Time: 04:01:37 ***  
Episode: 1091, Score: 32.93, Max: 32.93, Min: 32.93  
Episode: 1092, Score: 37.57, Max: 37.57, Min: 37.57  
Episode: 1093, Score: 25.14, Max: 25.14, Min: 25.14  
Episode: 1094, Score: 6.94, Max: 6.94, Min: 6.94  
Episode: 1095, Score: 14.19, Max: 14.19, Min: 14.19  
Episode: 1096, Score: 19.14, Max: 19.14, Min: 19.14  
Episode: 1097, Score: 34.80, Max: 34.80, Min: 34.80  
Episode: 1098, Score: 2.12, Max: 2.12, Min: 2.12  
Episode: 1099, Score: 18.13, Max: 18.13, Min: 18.13  
Episode: 1100, Score: 38.84, Max: 38.84, Min: 38.84  
*** Episode 1100	Average Score: 23.22, Time: 04:04:43 ***  
Episode: 1101, Score: 30.13, Max: 30.13, Min: 30.13  
Episode: 1102, Score: 26.36, Max: 26.36, Min: 26.36  
Episode: 1103, Score: 28.92, Max: 28.92, Min: 28.92  
Episode: 1104, Score: 33.17, Max: 33.17, Min: 33.17  
Episode: 1105, Score: 28.14, Max: 28.14, Min: 28.14  
Episode: 1106, Score: 22.16, Max: 22.16, Min: 22.16  
Episode: 1107, Score: 29.22, Max: 29.22, Min: 29.22  
Episode: 1108, Score: 15.65, Max: 15.65, Min: 15.65  
Episode: 1109, Score: 35.21, Max: 35.21, Min: 35.21  
Episode: 1110, Score: 29.66, Max: 29.66, Min: 29.66  
*** Episode 1110	Average Score: 23.56, Time: 04:07:49 ***  
Episode: 1111, Score: 28.25, Max: 28.25, Min: 28.25  
Episode: 1112, Score: 3.36, Max: 3.36, Min: 3.36  
Episode: 1113, Score: 29.60, Max: 29.60, Min: 29.60  
Episode: 1114, Score: 2.96, Max: 2.96, Min: 2.96  
Episode: 1115, Score: 30.15, Max: 30.15, Min: 30.15  
Episode: 1116, Score: 34.06, Max: 34.06, Min: 34.06  
Episode: 1117, Score: 2.57, Max: 2.57, Min: 2.57  
Episode: 1118, Score: 33.36, Max: 33.36, Min: 33.36  
Episode: 1119, Score: 8.94, Max: 8.94, Min: 8.94  
Episode: 1120, Score: 32.03, Max: 32.03, Min: 32.03  
*** Episode 1120	Average Score: 22.84, Time: 04:10:58 ***  
Episode: 1121, Score: 32.96, Max: 32.96, Min: 32.96  
Episode: 1122, Score: 37.52, Max: 37.52, Min: 37.52  
Episode: 1123, Score: 32.86, Max: 32.86, Min: 32.86  
Episode: 1124, Score: 22.38, Max: 22.38, Min: 22.38  
Episode: 1125, Score: 4.03, Max: 4.03, Min: 4.03  
Episode: 1126, Score: 34.08, Max: 34.08, Min: 34.08  
Episode: 1127, Score: 18.16, Max: 18.16, Min: 18.16  
Episode: 1128, Score: 33.31, Max: 33.31, Min: 33.31  
Episode: 1129, Score: 1.25, Max: 1.25, Min: 1.25  
Episode: 1130, Score: 0.56, Max: 0.56, Min: 0.56  
*** Episode 1130	Average Score: 23.41, Time: 04:14:08 ***  
Episode: 1131, Score: 30.80, Max: 30.80, Min: 30.80  
Episode: 1132, Score: 25.25, Max: 25.25, Min: 25.25  
Episode: 1133, Score: 31.19, Max: 31.19, Min: 31.19  
Episode: 1134, Score: 37.32, Max: 37.32, Min: 37.32  
Episode: 1135, Score: 36.79, Max: 36.79, Min: 36.79  
Episode: 1136, Score: 35.99, Max: 35.99, Min: 35.99  
Episode: 1137, Score: 35.10, Max: 35.10, Min: 35.10  
Episode: 1138, Score: 30.12, Max: 30.12, Min: 30.12  
Episode: 1139, Score: 23.30, Max: 23.30, Min: 23.30  
Episode: 1140, Score: 24.61, Max: 24.61, Min: 24.61   
*** Episode 1140	Average Score: 23.76, Time: 04:17:17 ***  
Episode: 1141, Score: 35.35, Max: 35.35, Min: 35.35   
Episode: 1142, Score: 17.12, Max: 17.12, Min: 17.12  
Episode: 1143, Score: 36.79, Max: 36.79, Min: 36.79  
Episode: 1144, Score: 29.94, Max: 29.94, Min: 29.94  
Episode: 1145, Score: 27.50, Max: 27.50, Min: 27.50  
Episode: 1146, Score: 27.41, Max: 27.41, Min: 27.41  
Episode: 1147, Score: 22.80, Max: 22.80, Min: 22.80  
Episode: 1148, Score: 29.17, Max: 29.17, Min: 29.17  
Episode: 1149, Score: 0.00, Max: 0.00, Min: 0.00  
Episode: 1150, Score: 23.87, Max: 23.87, Min: 23.87  
*** Episode 1150	Average Score: 23.82, Time: 04:20:27 ***  
Episode: 1151, Score: 35.24, Max: 35.24, Min: 35.24  
Episode: 1152, Score: 0.26, Max: 0.26, Min: 0.26  
Episode: 1153, Score: 29.35, Max: 29.35, Min: 29.35  
Episode: 1154, Score: 32.52, Max: 32.52, Min: 32.52  
Episode: 1155, Score: 23.62, Max: 23.62, Min: 23.62  
Episode: 1156, Score: 39.19, Max: 39.19, Min: 39.19  
Episode: 1157, Score: 34.60, Max: 34.60, Min: 34.60  
Episode: 1158, Score: 31.45, Max: 31.45, Min: 31.45  
Episode: 1159, Score: 23.93, Max: 23.93, Min: 23.93  
Episode: 1160, Score: 10.17, Max: 10.17, Min: 10.17  
*** Episode 1160	Average Score: 24.03, Time: 04:23:36 ***  
Episode: 1161, Score: 29.29, Max: 29.29, Min: 29.29   
Episode: 1162, Score: 38.10, Max: 38.10, Min: 38.10  
Episode: 1163, Score: 36.91, Max: 36.91, Min: 36.91  
Episode: 1164, Score: 29.42, Max: 29.42, Min: 29.42  
Episode: 1165, Score: 36.00, Max: 36.00, Min: 36.00  
Episode: 1166, Score: 30.77, Max: 30.77, Min: 30.77   
Episode: 1167, Score: 5.10, Max: 5.10, Min: 5.10  
Episode: 1168, Score: 20.32, Max: 20.32, Min: 20.32  
Episode: 1169, Score: 37.60, Max: 37.60, Min: 37.60  
Episode: 1170, Score: 23.36, Max: 23.36, Min: 23.36  
*** Episode 1170	Average Score: 24.73, Time: 04:26:45 ***  
Episode: 1171, Score: 8.14, Max: 8.14, Min: 8.14  
Episode: 1172, Score: 11.27, Max: 11.27, Min: 11.27   
Episode: 1173, Score: 37.07, Max: 37.07, Min: 37.07  
Episode: 1174, Score: 36.09, Max: 36.09, Min: 36.09  
Episode: 1175, Score: 30.73, Max: 30.73, Min: 30.73  
Episode: 1176, Score: 31.25, Max: 31.25, Min: 31.25  
Episode: 1177, Score: 35.81, Max: 35.81, Min: 35.81  
Episode: 1178, Score: 37.71, Max: 37.71, Min: 37.71  
Episode: 1179, Score: 36.87, Max: 36.87, Min: 36.87  
Episode: 1180, Score: 39.61, Max: 39.61, Min: 39.61   
*** Episode 1180	Average Score: 25.44, Time: 04:29:54 ***  
Episode: 1181, Score: 33.73, Max: 33.73, Min: 33.73  
Episode: 1182, Score: 36.59, Max: 36.59, Min: 36.59  
Episode: 1183, Score: 36.90, Max: 36.90, Min: 36.90  
Episode: 1184, Score: 33.51, Max: 33.51, Min: 33.51  
Episode: 1185, Score: 32.91, Max: 32.91, Min: 32.91  
Episode: 1186, Score: 32.95, Max: 32.95, Min: 32.95  
Episode: 1187, Score: 35.45, Max: 35.45, Min: 35.45  
Episode: 1188, Score: 34.91, Max: 34.91, Min: 34.91  
Episode: 1189, Score: 36.91, Max: 36.91, Min: 36.91  
Episode: 1190, Score: 39.46, Max: 39.46, Min: 39.46  
*** Episode 1190	Average Score: 26.96, Time: 04:33:03 ***  
Episode: 1191, Score: 36.23, Max: 36.23, Min: 36.23  
Episode: 1192, Score: 36.38, Max: 36.38, Min: 36.38  
Episode: 1193, Score: 25.89, Max: 25.89, Min: 25.89  
Episode: 1194, Score: 36.86, Max: 36.86, Min: 36.86  
Episode: 1195, Score: 37.01, Max: 37.01, Min: 37.01  
Episode: 1196, Score: 15.57, Max: 15.57, Min: 15.57  
Episode: 1197, Score: 25.43, Max: 25.43, Min: 25.43  
Episode: 1198, Score: 27.80, Max: 27.80, Min: 27.80  
Episode: 1199, Score: 6.07, Max: 6.07, Min: 6.07  
Episode: 1200, Score: 6.96, Max: 6.96, Min: 6.96  
*** Episode 1200	Average Score: 27.21, Time: 04:36:13 ***  
Episode: 1201, Score: 34.18, Max: 34.18, Min: 34.18  
Episode: 1202, Score: 39.23, Max: 39.23, Min: 39.23  
Episode: 1203, Score: 30.70, Max: 30.70, Min: 30.70  
Episode: 1204, Score: 11.29, Max: 11.29, Min: 11.29  
Episode: 1205, Score: 38.58, Max: 38.58, Min: 38.58  
Episode: 1206, Score: 25.18, Max: 25.18, Min: 25.18  
Episode: 1207, Score: 38.91, Max: 38.91, Min: 38.91  
Episode: 1208, Score: 30.44, Max: 30.44, Min: 30.44  
Episode: 1209, Score: 33.23, Max: 33.23, Min: 33.23  
Episode: 1210, Score: 38.46, Max: 38.46, Min: 38.46  
*** Episode 1210	Average Score: 27.62, Time: 04:39:22 ***  
Episode: 1211, Score: 28.28, Max: 28.28, Min: 28.28  
Episode: 1212, Score: 37.52, Max: 37.52, Min: 37.52  
Episode: 1213, Score: 39.32, Max: 39.32, Min: 39.32  
Episode: 1214, Score: 39.51, Max: 39.51, Min: 39.51  
Episode: 1215, Score: 30.80, Max: 30.80, Min: 30.80  
Episode: 1216, Score: 27.62, Max: 27.62, Min: 27.62  
Episode: 1217, Score: 37.91, Max: 37.91, Min: 37.91  
Episode: 1218, Score: 39.27, Max: 39.27, Min: 39.27  
Episode: 1219, Score: 37.20, Max: 37.20, Min: 37.20  
Episode: 1220, Score: 31.23, Max: 31.23, Min: 31.23  
*** Episode 1220	Average Score: 29.06, Time: 04:42:32 ***  
Episode: 1221, Score: 37.47, Max: 37.47, Min: 37.47  
Episode: 1222, Score: 28.28, Max: 28.28, Min: 28.28  
Episode: 1223, Score: 30.29, Max: 30.29, Min: 30.29  
Episode: 1224, Score: 39.40, Max: 39.40, Min: 39.40  
Episode: 1225, Score: 35.79, Max: 35.79, Min: 35.79  
Episode: 1226, Score: 21.43, Max: 21.43, Min: 21.43  
Episode: 1227, Score: 39.51, Max: 39.51, Min: 39.51  
Episode: 1228, Score: 26.49, Max: 26.49, Min: 26.49  
Episode: 1229, Score: 38.56, Max: 38.56, Min: 38.56  
Episode: 1230, Score: 37.95, Max: 37.95, Min: 37.95  
*** Episode 1230	Average Score: 30.24, Time: 04:45:42 ***  
Environment solved ! 

### Future Ideas for Improving

1. The improvement can be achieved by adding more layers to the neural networks Actor and Critic, making them more deeper. According to a paper called     
   that Batch Normalization can accelerate Deep Network Training, 
   for example, [here](https://medium.com/@ilango100/batch-normalization-speed-up-neural-network-training-245e39a62f85) and [here](https://arxiv.org/pdf/1502.03167.pdf).

2. Tune the hyperparameters such as BATCH_SIZE, LR_ACTOR,  LR_CRITIC, LEARNING_PERIOD, UPDATE_FACTOR with other values.    
 
3. Instead of DDPG, other models can be considered, such as [PPO](https://openai.com/blog/openai-baselines-ppo/), 
   [A3C](https://blog.goodaudience.com/a3c-what-it-is-what-i-built-6b91fe5ec09c) and others.